## 카드 회원정보 리딩


금융합성데이터
01.카드 회원정보 : 201807_회원정보부터 201812_회원정보.csv
02.카드 신용정보 : 201807_신용정보부터 201812_신용정보.csv
03.카드 승인매출정보 : 201807_승인매출정보부터 201812_승인매출정보.csv
04.카드 청구정보 : 201807_청구정보부터 201812_청구정보.csv
05.카드 잔액정보 : 201807_잔액정보부터 201812_잔액정보.csv
06.카드 채널정보 : 201807_채널정보부터 201812_채널정보.csv
07.카드 마케팅정보 : 201807_마케팅정보부터 201812_마케팅정보.csv
08.카드 성과정보 : 201807_부터 201812_성과정보.csv
09.개인 CB정보 : 201912_개인CB부터 202212_개인CB.csv
10.기업 CB정보 :
11.통신카드CB 결합정보
12.금융상품정보

In [0]:
import glob
import pandas as pd
import os

In [0]:

dbutils.fs.ls("/mnt/raw-data/01.카드 회원정보/")


In [0]:
df1 = spark.read.csv("/mnt/raw-data/01.카드 회원정보/2018*_회원정보.csv", header=True,encoding='utf-8')
df1


In [0]:
df1 = spark.read.csv("/mnt/raw-data/01.카드 회원정보/2018*_회원정보.csv", header=True, encoding='utf-8')


In [0]:
df1.printSchema()


In [0]:
display(df1.limit(2))


In [0]:
df1.limit(2).toPandas()


In [0]:

columns_info = pd.DataFrame({
    "Column Name": df1.columns,
    "Unique Values": [df1.select(col).distinct().count() for col in df1.columns],
    "Data Type": df1.dtypes,
    "Null Values": df1.isnull().sum(),

    "Example Value": [df1[col].sample(1).values[0] for col in df1.columns]

})
columns_info.reset_index(drop=True, inplace=True)

In [0]:
df1.printSchema()  
display(df1.limit(3))


In [0]:
display(df1.head(3))
#display(df_credit.tail(3))
#display(df_credit.sample(3))

# 데이터 탐색 보고서 (EDA)

##  데이터 개요
- **파일명**: `1번_고객기본정보.csv`, `8번_전월이용증감률.csv`
- **총 행 수 / 열 수**: `예: 98,412 / 12`
- **수집 일자 범위**: `2021.01 ~ 2024.12 (예시)`
- **데이터 목적**: `예: 고객의 카드 이용 특성과 이탈 패턴 분석`

---

##  1. 데이터 구조 및 기본 정보 확인
- 컬럼 목록:  
  `df.columns`
- 스키마 구조:  
  `df.printSchema()`
- 상위 5개 행:
  ```
  df.limit(5).display()
  ```
- 변수별 데이터 타입:
  ```
  df.dtypes
  ```

---

##  2. 결측치 및 이상치 확인
###  결측치 확인
```python
from pyspark.sql.functions import col, sum
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).display()
```
- 결측치가 많은 변수: `예: 최종카드발급일자 (22%)`
- 처리 방향성: `삭제 / 평균 대체 / 별도 범주 처리 등`

###  이상치 탐색 (IQR 기준)
```python
Q1 = df.approxQuantile("변수명", [0.25], 0.05)[0]
Q3 = df.approxQuantile("변수명", [0.75], 0.05)[0]
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
df.filter((col("변수명") < lower) | (col("변수명") > upper)).count()
```
- 이상치 발견된 변수: `예: 이용금액_R3M_신용`
- 처리 계획: `클리핑 / 로그 변환`

---

##  3. 변수 유형 및 그룹 분류
- **범주형 변수**: `성별, 연령대, 카드구분`
- **수치형 변수**: `이용금액, 경과개월수, 이용건수`
- **날짜/시계열**: `최종카드발급일자, 최종유효년월`
- **그룹화 변수**: `_1순위카드이용건수`, `_1순위신용체크구분`

---

##  4. 변수 간 관계 파악
###  상관계수 (수치형 간)
```python
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

assembler = VectorAssembler(inputCols=["변수1", "변수2", ...], outputCol="features")
df_vector = assembler.transform(df).select("features")
correlation_matrix = Correlation.corr(df_vector, "features", "pearson").head()
```
- 예시: `_1순위카드이용금액` ↔ `이용금액_R3M_신용` 상관계수 = `0.81`

---

##  5. 데이터 분포 파악
- 범주형 변수 분포 (Top 10):
```python
df.groupBy("범주형변수").count().orderBy("count", ascending=False).limit(10).display()
```
- 수치형 변수 히스토그램:
```python
df.select("변수명").toPandas().hist()
```
- 로그 스케일 비교 (선택):
```python
import numpy as np
df = df.withColumn("log_이용금액", F.log1p("이용금액_R3M_신용"))
```

---

##  6. 전처리 방향성 정리
| 항목 | 처리 방향 |
|------|-----------|
| 결측치 | `최종카드발급일자`: NULL이면 '미발급'으로 처리 |
| 이상치 | `이용금액`: 상위 1% 클리핑 또는 로그 변환 |
| 인코딩 | 범주형 변수: StringIndexer 또는 수동 매핑 |
| 변수 선택 | 모델링 핵심 변수 후보: `이용금액_R3M_신용`, `_1순위카드이용금액`, `성별` 등 |

---

##  요약
- 데이터는 고객의 기본 정보 및 카드 사용 패턴 중심
- 최근 3개월 이용금액, 1순위 카드 변수에서 유의미한 분포 및 상관관계 존재
- 향후 모델링 시 고객 세분화 및 이탈 예측에 활용 가능성 있음



## 1번 파일 데이터 이해

### 📌 기본 정보
- 성별
- 연령대
- 거주지
- 직장 위치 등

### 📌 회원 상태
- 입회일
- 탈퇴 이력
- 연체 여부 등

### 📌 카드 보유 현황
- 신용/체크카드 보유 여부 및 수량

### 📌 카드 이용 이력
- 최근 3개월 카드 사용금액 및 건수

### 📌 마케팅 수신 여부
- DM, SMS, TM 등

### 📌 기타
- VIP 등급
- 카드 등급
- Life Stage 등

---

## 📄 Row 예시 1 (발급회원번호 기준)

| 항목 | 설명 |
|------|------|
| 기준년월 | 201807 (2018년 7월 기준 데이터) |
| 성별 | 2 (여성) |
| 연령대 | 40대 |
| VIP등급 | 07 |
| 입회일자 | 2013-01-01 (입회 후 67개월 경과) |
| 최종카드발급일자 | 2016-09-12 (발급 후 22개월 경과) |
| 신용카드 소지 | 1장 (이용 가능), 가족카드 없음 |
| 체크카드 소지 | 1장 (이용 불가), 가족카드 없음 |
| 최근 3개월 신용카드 이용금액 | 19,612원 |
| 1순위 카드 이용금액 | 368,148원 (26건, 신용카드) |
| 최종 탈회 후 경과월 | 61개월 |
| 마케팅 수신 여부 | SMS/메일/TM 모두 수신 |
| Life Stage | 5.자녀성장기(2) |
| 거주/직장 시도 | 경기 / 경기 |

---

## 📄 Row 예시 2 — 발급회원번호 = `SYN_1`

| 항목 | 설명 |
|------|------|
| 기준년월 | 201807 (2018년 7월 기준 데이터) |
| 성별 | 1 (남성) |
| 연령대 | 30대 |
| VIP등급 | 없음 |
| 입회일자 | 2017-08-01 (입회 후 12개월 경과) |
| 최종카드발급일자 | 2017-01-22 (발급 후 18개월 경과) |
| 신용카드 소지 | 1장 (이용 가능), 가족카드 없음 |
| 체크카드 소지 | 없음 |
| 최근 3개월 신용카드 이용금액 | 1,347,574원 |
| 1순위 카드 이용금액 | 1,332,368원 (46건, 신용카드) |
| 최종 탈회 후 경과월 | 98개월 |
| 마케팅 수신 여부 | SMS/메일/TM 모두 수신 |
| Life Stage | 4.자녀성장기(1) |
| 거주/직장 시도 | 서울 / 서울 |


소비패턴 차이 확인 가능
체크카드 미사용 고객 존재
탈퇴 후 재 입회 고객 존재(재가입 이력 확인)


고객군 나눠서 소비 금액별 분류, vip등급 특징, 연령대 이용 패턴정도 확인?

## 결측치 및 이상치 확인 파트

In [0]:
df1.printSchema()


In [0]:
from pyspark.sql.functions import col, isnan, when, count

# 각 컬럼별 결측치 수 계산
missing_df1 = df1.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in df1.columns
])

# 전체 row 수
total_rows1 = df1.count()

# 비율 포함해 보기 좋게 변환
missing_pd1 = missing_df1.toPandas().T
missing_pd1.columns = ['결측치 수']
missing_pd1['전체 대비 비율 (%)'] = (missing_pd1['결측치 수'] / total_rows1) * 100
missing_pd1 = missing_pd1[missing_pd1['결측치 수'] > 0].sort_values(by='결측치 수', ascending=False)

display(missing_pd1)


In [0]:
# 컬럼별 결측치 수와 비율을 데이터프레임으로 변환 후 컬럼명 추가
missing_pd1 = missing_df1.toPandas().T
missing_pd1.columns = ['결측치 수']
missing_pd1['전체 대비 비율 (%)'] = (missing_pd1['결측치 수'] / total_rows1) * 100
missing_pd1['컬럼명'] = missing_pd1.index
missing_pd1 = missing_pd1[missing_pd1['결측치 수'] > 0].sort_values(by='결측치 수', ascending=False)

display(missing_pd1)


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 그래프 스타일 설정
plt.figure(figsize=(12, 6))
colors = sns.color_palette("hls", len(missing_pd1))  # 항목별 색상 지정

# 막대 그래프 그리기
bars = plt.bar(missing_pd1['컬럼명'], missing_pd1['전체 대비 비율 (%)'], color=colors)

# 수치 표시
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2,
             height + 0.5,
             f'{height:.1f}%',
             ha='center',
             va='bottom',
             fontsize=9)

# 그래프 제목 및 축 라벨
plt.title('컬럼별 결측치 비율 (%)', fontsize=14)
plt.xlabel('컬럼명', fontsize=12)
plt.ylabel('전체 대비 비율 (%)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, max(missing_pd1['전체 대비 비율 (%)']) * 1.15)
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()


| 결측치 수   | 전체 대비 비율 (%)          | 컬럼명               |
|-------------|----------------------------|----------------------|
| 7,189,389   | 39.94105                   | _2순위신용체크구분    |
| 4,031,543   | 22.39746                   | 최종유효년월_신용_이용 |
| 2,910,604   | 16.17002                   | 가입통신회사코드       |
| 1,829,715   | 10.16508                   | 직장시도명            |
| 1,590,648   | 8.83693                    | 최종유효년월_신용_이용가능 |
| 319,362     | 1.77423                    | 최종카드발급일자       |
| 211,517     | 1.17509                    | _1순위신용체크구분     |

결측 비율 상위 7개인데 4번까지는 제하고 분석하거나 따로 처리 요망?

## 이상치(수치형)

In [0]:
from pyspark.sql.functions import col

def detect_outliers_iqr(df, column):
    quantiles = df.approxQuantile(column, [0.25, 0.75], 0.01)
    Q1, Q3 = quantiles[0], quantiles[1]
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df.filter((col(column) < lower_bound) | (col(column) > upper_bound))
    count_outliers = outliers.count()
    total_count = df.count()
    outlier_ratio = (count_outliers / total_count) * 100

    return {
        'column': column,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'outlier_count': count_outliers,
        'outlier_ratio(%)': outlier_ratio
    }


In [0]:
from pyspark.sql.functions import col

# 숫자형으로 변환 (변환 불가능한 값은 null 처리됨) 이용금액_R3M_신용이 StringType이라서 수치형 연산 안됨.
for col_name in columns_to_check:
    df1 = df1.withColumn(col_name, col(col_name).cast('double'))


In [0]:
columns_to_check = [
    '입회경과개월수_신용', '유효카드수_신용체크', '이용가능카드수_신용체크', '이용카드수_신용체크',
    '이용금액_R3M_신용체크', '기본연회비_B0M', '_1순위카드이용금액'
]

results = []
for col_name in columns_to_check:
    results.append(detect_outliers_iqr(df1, col_name))

import pandas as pd
outlier_summary = pd.DataFrame(results)
display(outlier_summary)


| column               | lower_bound  | upper_bound  | outlier_count | outlier_ratio (%)       |
|----------------------|--------------|--------------|---------------|-------------------------|
| 이용금액_R3M_신용     | -2914045.5   | 4856742.5    | 1497865       | 8.321472222222221       |
| 이용금액_R3M_체크     | 0            | 0            | 2835580       | 15.753222222222222      |
| _1순위카드이용금액     | -2371819.5   | 3953032.5    | 954797        | 5.304427777777778       |
| _2순위카드이용금액     | -444004.5    | 740007.5     | 2905656       | 16.142533333333333      |
| _1순위카드이용건수     | -89          | 151          | 1025805       | 5.698916666666666       |
| _2순위카드이용건수     | -22.5        | 37.5         | 2704890       | 15.027166666666666      |
| 최종카드발급경과월     | -19          | 61           | 1065          | 0.005916666666666666    |
| 입회경과개월수_신용    | -110.5       | 229.5        | 1066025       | 5.922361111111111       |
| 탈회횟수_누적          | -1.5         | 2.5          | 2             | 0.000011111111111111112 |
| 청구금액_기본연회비_B0M | 0            | 0            | 22277         | 0.12376111111111113     |
| 청구금액_제휴연회비_B0M | 0            | 0            | 6626          | 0.03681111111111111     |
| 카드신청건수           | 0            | 0            | 1577722       | 8.765122222222223       |


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

columns_to_plot = [
    '이용금액_R3M_신용',
    '이용금액_R3M_체크',
    '_1순위카드이용금액',
    '_2순위카드이용금액',
    '_1순위카드이용건수',
    '_2순위카드이용건수',
    '최종카드발급경과월',
    '입회경과개월수_신용'
]

# 필요한 컬럼만 선택해서 Pandas DataFrame으로 변환 (샘플링 권장)
pdf1 = df1.select(columns_to_plot).sample(fraction=0.1, seed=42).toPandas()

for col in columns_to_plot:
    pdf1[col] = pd.to_numeric(pdf1[col], errors='coerce')


In [0]:
# 수치형 컬럼 추정 몇개 선택한거
pdf1

In [0]:
#한글
!apt-get update -qq
!apt-get install -y fonts-nanum

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
 
font_dirs = ["/usr/share/fonts/truetype/nanum/"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
 
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
 
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)
 
pd.Series([-1,2,3]).plot(title='테스트', figsize=(3,2))
pass

In [0]:


plt.figure(figsize=(15, 8))
sns.boxplot(data=pdf1, orient='h')
plt.show()


1번 이용금액_r3m_신용 보면 상당수 이상치 우측 밀집, 평균 혹은 중앙값보다 훨씬 큰값 존재 -> 소비액이 극단적으로 큰 일부 고객군 존재

2번 이용금액-r3m_체크도 똑같다.

3번 _1순위카드이용금액은 이상치가 있긴한데 적고, 극단적 값만 관찰정도

4번_2순위카드이용금액은 이상치가 왼쪽 올느쪽 골곡루 있다. 대부분 낮은값임

나머지는 이상치 x 분포 안정적

이라고 ai는 해석하는데 사람눈으로 보는거랑 실제 그래프랑 다른가보다.

## 변수 유형 및 그룹 분류

In [0]:
# 컬럼명 출력
print(df1.columns)

# 데이터 타입 확인 (PySpark 기준)
print(df1.dtypes)

# 상위 5개 행 샘플 출력
df1.show(5)


확인결과 74개 컬럼

문자열(string)다수, 수치형(double) 일부

자동화 코드 해서 컬럼별로 다하는게 가능은하나 힘들고 메모리 많이 잡아먹나?

일단 13개 기준으로 하려고 하는중
남녀구분

연령대/입회경과개월수_신용//최종탈회후경과월/이용금액_R3M_신용체크/이용금액_R3M_신용/_1순위카드이용금액/_1순위카드이용건수/_1순위신용체크구분/최종유효년월_신용_이용가능/최종유효년월_신용_이용/최종카드발급일자/최종카드발급경과월

여기서 한버 ㄴ74개 분류 시도

In [0]:
#날짜변환하고
from pyspark.sql.functions import to_date

df1 = df1.withColumn("입회일자_신용", to_date("입회일자_신용", "yyyyMMdd"))
df1 = df1.withColumn("최종카드발급일자", to_date("최종카드발급일자", "yyyyMMdd"))


In [0]:
from pyspark.sql.types import StringType, NumericType, DateType, TimestampType

categorical_vars = []
numerical_vars = []
date_vars = []

for field in df1.schema.fields:
    col_name = field.name
    dtype = field.dataType

    if isinstance(dtype, (DateType, TimestampType)):
        date_vars.append(col_name)
    elif isinstance(dtype, NumericType):
        numerical_vars.append(col_name)
    elif isinstance(dtype, StringType):
        categorical_vars.append(col_name)
    else:
        # 필요시 다른 타입도 범주형으로 분류 가능
        categorical_vars.append(col_name)

print("범주형 변수:", categorical_vars)
print("수치형 변수:", numerical_vars)
print("날짜 변수:", date_vars)


그냥 74개 행 전부다 해버리는게 나을거같다

In [0]:
from pyspark.sql import functions as F

# 각 컬럼별 결측치 개수 확인
df1.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df1.columns]).show(truncate=False)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row

total_count = df1.count()

# 각 컬럼별 결측치 개수 계산
null_counts = df1.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df1.columns]).collect()[0].asDict()

# 컬럼명과 결측치 개수를 리스트로 변환
null_list = [(col, cnt, cnt / total_count * 100) for col, cnt in null_counts.items()]

# PySpark DataFrame으로 변환
null_df = spark.createDataFrame([Row(컬럼명=col, 결측치_개수=cnt, 결측치_비율=f"{ratio:.2f}%") for col, cnt, ratio in null_list])

# 결측치 개수 기준 내림차순 정렬해서 보기 좋게 출력
null_df.orderBy(F.desc("결측치_개수")).show(truncate=False)


# 📘 변수 설명 리스트 (총 74개)

## 🟦 범주형 변수 (Categorical Variables)

| 변수명 | 설명 |
|--------|------|
| 기준년월 | 데이터 기준 연월 (YYYYMM) |
| 발급회원번호 | 고객 식별용 고유번호 |
| 남녀구분코드 | 성별 구분 (남/여) |
| 연령 | 연령대 구분 (20대, 30대 등) |
| VIP등급코드 | 고객의 VIP 등급 코드 |
| 최상위카드등급코드 | 보유 카드 중 가장 높은 등급 |
| 회원여부_이용가능 | 해당 월 기준 회원 여부 |
| 회원여부_이용가능_CA | CA상품 기준 회원 여부 |
| 회원여부_이용가능_카드론 | 카드론 기준 회원 여부 |
| 소지여부_신용 | 신용카드 소지 여부 |
| 소지카드수_유효_신용 | 유효한 신용카드 수 |
| 소지카드수_이용가능_신용 | 이용 가능한 신용카드 수 |
| 회원여부_연체 | 연체 이력 여부 |
| 이용거절여부_카드론 | 카드론 이용 거절 여부 |
| 동의여부_한도증액안내 | 한도 증액 안내 동의 여부 |
| 수신거부여부_TM | 텔레마케팅 수신 거부 여부 |
| 수신거부여부_DM | DM(우편광고) 수신 거부 여부 |
| 수신거부여부_메일 | 이메일 수신 거부 여부 |
| 수신거부여부_SMS | 문자 수신 거부 여부 |
|_ 가입통신회사코드 | 가입된 통신사 코드,결측치 16.17% |_
| 최종탈회후경과월 | 마지막 탈회 후 경과 개월 수 |
| 탈회횟수_발급6개월이내 | 카드 발급 후 6개월 이내 탈회 횟수 |
| 탈회횟수_발급1년이내 | 카드 발급 후 1년 이내 탈회 횟수 |
| 거주시도명 | 고객 거주지의 시도명 |
| 직장시도명 | 직장 위치의 시도명,결측치 10.17% |
| 마케팅동의여부 | 마케팅 수신 동의 여부 |
| 유효카드수_신용체크 | 유효한 신용/체크카드 수 (전체) |
| 유효카드수_신용 | 유효한 신용카드 수 (본인) |
| 유효카드수_신용_가족 | 유효한 신용카드 수 (가족) |
| 유효카드수_체크 | 유효한 체크카드 수 (본인) |
| 유효카드수_체크_가족 | 유효한 체크카드 수 (가족) |
| 이용가능카드수_신용체크 | 이용 가능한 신용/체크카드 수 (전체) |
| 이용가능카드수_신용 | 이용 가능한 신용카드 수 (본인) |
| 이용가능카드수_신용_가족 | 이용 가능한 신용카드 수 (가족) |
| 이용가능카드수_체크 | 이용 가능한 체크카드 수 (본인) |
| 이용가능카드수_체크_가족 | 이용 가능한 체크카드 수 (가족) |
| 이용카드수_신용체크 | 최근 사용한 신용/체크카드 수 |
| 이용카드수_신용 | 최근 사용한 신용카드 수 (본인) |
| 이용카드수_신용_가족 | 최근 사용한 신용카드 수 (가족) |
| 이용카드수_체크 | 최근 사용한 체크카드 수 (본인) |
| 이용카드수_체크_가족 | 최근 사용한 체크카드 수 (가족) |
| 이용금액_R3M_신용체크 | 최근 3개월간 신용/체크 이용금액 |
| 이용금액_R3M_신용_가족 | 최근 3개월간 신용카드 이용금액 (가족) |
| 이용금액_R3M_체크_가족 | 최근 3개월간 체크카드 이용금액 (가족) |
| _1순위신용체크구분 | 가장 많이 사용한 카드 종류,결측치 1.18%|
| _2순위신용체크구분 | 두 번째로 많이 사용한 카드 종류, 결측치 39.94% |
| 최종유효년월_신용_이용가능 | 마지막 이용 가능한 월 (신용),결측치 8.84 |
| 최종유효년월_신용_이용 | 마지막 실제 사용한 월 (신용),결측치 22.4% |
| 보유여부_해외겸용_본인 | 해외겸용카드 보유 여부 (본인 전체) |
| 이용가능여부_해외겸용_본인 | 해외겸용카드 사용 가능 여부 (본인 전체) |
| 이용여부_3M_해외겸용_본인 | 최근 3개월간 해외겸용카드 사용 여부 |
| 보유여부_해외겸용_신용_본인 | 해외겸용 신용카드 보유 여부 |
| 이용가능여부_해외겸용_신용_본인 | 해외겸용 신용카드 사용 가능 여부 |
| 이용여부_3M_해외겸용_신용_본인 | 최근 3개월간 해외겸용 신용카드 사용 여부 |
| 연회비발생카드수_B0M | 연회비가 발생한 카드 수 |
| 연회비할인카드수_B0M | 연회비 할인 받은 카드 수 |
| 기본연회비_B0M | 기본 연회비 설정 금액 |
| 제휴연회비_B0M | 제휴 연회비 설정 금액 |
| 할인금액_기본연회비_B0M | 기본 연회비 할인 금액 |
| 할인금액_제휴연회비_B0M | 제휴 연회비 할인 금액 |
| 상품관련면제카드수_B0M | 상품 혜택으로 연회비 면제 카드 수 |
| 임직원면제카드수_B0M | 임직원 혜택으로 연회비 면제 카드 수 |
| 우수회원면제카드수_B0M | 우수회원 혜택으로 연회비 면제 카드 수 |
| 기타면제카드수_B0M | 기타 사유로 면제된 카드 수 |
| Life_Stage | 생애주기 구분 (예: 사회초년생, 중년 등) |

---

## 🟩 수치형 변수 (Numerical Variables)

| 변수명 | 설명 |
|--------|------|
| 입회경과개월수_신용 | 신용카드 입회 후 경과 개월 수 |
| 탈회횟수_누적 | 누적 탈회 횟수 |
| 이용금액_R3M_신용 | 최근 3개월간 신용카드 이용금액 |
| 이용금액_R3M_체크 | 최근 3개월간 체크카드 이용금액 |
| _1순위카드이용금액 | 가장 많이 사용한 카드의 이용금액 |
| _1순위카드이용건수 | 가장 많이 사용한 카드의 이용건수 |
| _2순위카드이용금액 | 두 번째로 많이 사용한 카드의 이용금액 |
| _2순위카드이용건수 | 두 번째로 많이 사용한 카드의 이용건수 |
| 청구금액_기본연회비_B0M | 실제 청구된 기본 연회비 |
| 청구금액_제휴연회비_B0M | 실제 청구된 제휴 연회비 |
| 카드신청건수 | 누적 카드 신청 건수 |
| 최종카드발급경과월 | 마지막 카드 발급 후 경과 개월 수 |

---

## 🟨 날짜형 변수 (Date Variables)

| 변수명 | 설명 |
|--------|------|
| 입회일자_신용 | 신용카드 가입 일자 |
| 최종카드발급일자 | 마지막 카드 발급 일자,결측치 1.17% |

## 🧪 결측치 비율

| 컬럼명                      | 결측치 개수 | 결측치 비율 |
|----------------------------|--------------|--------------|
| _2순위신용체크구분         | 7,189,389    | 39.94%       |
| 최종유효년월_신용_이용     | 4,031,543    | 22.40%       |
| 가입통신회사코드           | 2,910,604    | 16.17%       |
| 직장시도명                 | 1,829,715    | 10.17%       |
| 최종유효년월_신용_이용가능 | 1,590,648    | 8.84%        |
| 최종카드발급일자           | 319,362      | 1.77%        |
| _1순위신용체크구분         | 211,517      | 1.18%        |



| 컬럼명 | 결측치 개수 | 결측치  비율 | 결측치 처리 방식 |
|---|---|-----|-----|
| _2순위신용체크구분 | 7,189,389 | 39.94% | "없음"으로 대체(현금사용 or 1순위카드만사용) |
| 최종유효년월_신용_이용 | 4,031,543 | 22.40% | "없음"으로대체(현금유저), 날짜형식 범주형으로 변환 요망 |
| 가입통신회사코드 | 2,910,604 | 16.17% | "알뜰폰"으로 대체 |
| 직장시도명 | 1,829,715 | 10.17% | "비경제"대체 |
| 최종유효년월_신용_이용가능 | 1,590,648 | 8.84% | "없음"으로대체(현금유저),날짜형식 범주형으로 변환 요망 |
| 최종카드발급일자 | 319,362 | 1.77% |  행 삭제 |
| _1순위신용체크구분 | 211,517 | 1.18% | "현금"으로대체(현금유저) |

어떠한 변수를 사용할것인가???

13개중 말고 74개중

분석 목적과 직접 관련 있는 변수
예: 이용금액_R3M_신용, 입회경과개월수_신용, VIP등급코드, 최종탈회후경과월 등.

시간 흐름을 나타내는 지표
예: 최종카드발급경과월, 최종유효년월_신용_이용, 입회일자_신용

활성/비활성 고객을 나눌 수 있는 변수
예: 이용카드수_신용, 이용가능카드수_신용, 수신거부여부_SMS, 회원여부_연체

잠재 이탈 예측에 활용할 수 있는 지표
예: 탈회횟수_누적, 회원여부_이용가능, 이용금액_R3M_체크

아니면 상관계수 분석이나
변수 중요도 판단(gini importance)
혹은 피쳐 중요도 시각화로 해서 한다.

10~20개 수작업하기?


### ERD 준비
erd로 컬럼을 모으기로했음

##1. 회원정보 테이블
[회원정보]
- 회원ID (PK)
- 성별
- 연령대
- 입회일자_신용
- 입회경과개월수_신용
- 직장시도명
- 가입통신회사코드
- VIP등급코드
- 수신거부여부_SMS
- 수신거부여부_DM
- 수신거부여부_TMK

##2. 카드이용정보 테이블
[카드이용정보]
- 회원ID (FK)
- 이용금액_R3M_신용
- 이용금액_R3M_체크
- 이용건수_R3M_신용
- 이용건수_R3M_체크
- _1순위카드이용금액
- _1순위카드이용건수
- _1순위신용체크구분
- _2순위신용체크구분


##3. 카드상태 테이블
[카드상태]
- 회원ID (FK)
- 이용카드수_신용
- 이용가능카드수_신용
- 최종유효년월_신용_이용
- 최종유효년월_신용_이용가능
- 최종카드발급일자
- 최종카드발급경과월



##4. 회원상태 테이블
[회원상태]
- 회원ID (FK)
- 회원여부_이용가능
- 회원여부_연체
- 탈회횟수_누적
- 최종탈회후경과월




##5. 이용패턴파생 테이블
[이용패턴파생]
- 회원ID (FK)
- 증감율_이용건수_신용_전월
- 증감율_이용건수_신판_전월
- 증감율_이용건수_일시불_전월





Table 회원정보 {
  회원ID int PK
  성별 string
  연령대 string
  입회일자_신용 date
  입회경과개월수_신용 int
  직장시도명 string
  가입통신회사코드 string
  VIP등급코드 string
  수신거부여부_SMS bool
  수신거부여부_DM bool
  수신거부여부_TMK bool
}

Table 카드이용정보 {
  회원ID int FK
  이용금액_R3M_신용 float
  이용금액_R3M_체크 float
  이용건수_R3M_신용 int
  이용건수_R3M_체크 int
  _1순위카드이용금액 float
  _1순위카드이용건수 int
  _1순위신용체크구분 string
  _2순위신용체크구분 string
}

Table 카드상태 {
  회원ID int FK
  이용카드수_신용 int
  이용가능카드수_신용 int
  최종유효년월_신용_이용 string
  최종유효년월_신용_이용가능 string
  최종카드발급일자 date
  최종카드발급경과월 int
}

Table 회원상태 {
  회원ID int FK
  회원여부_이용가능 bool
  회원여부_연체 bool
  탈회횟수_누적 int
  최종탈회후경과월 int
}

Table 이용패턴파생 {
  회원ID int FK
  증감율_이용건수_신용_전월 float
  증감율_이용건수_신판_전월 float
  증감율_이용건수_일시불_전월 float
}

Ref: 카드이용정보.회원ID > 회원정보.회원ID  
Ref: 카드상태.회원ID > 회원정보.회원ID  
Ref: 회원상태.회원ID > 회원정보.회원ID  
Ref: 이용패턴파생.회원ID > 회원정보.회원ID


In [0]:
df1.select("Life_Stage").distinct().limit(30).display()


In [0]:
df1.select("_2순위신용체크구분").distinct().limit(30).display()



In [0]:
df1.select("가입통신회사코드").distinct().limit(30).display()

In [0]:
df1.select("_1순위신용체크구분").distinct().limit(30).display()



_2순위신용체크부분 결측치와 이용카드 수 신용체크 관계
0,1개 고객이 전체 결측치 거의 전부,
즉 2순위 신용체크 부분 결측치는 단일카드 보유자로 판단.

In [0]:
# 1순위 카드 결측은 1.18? 2순위는 39.94, MSAR보다 MNAR 생각, 40퍼가 읭답오류? 이거보단 현금쓰는 양반 1.18미만에다가 카드 하나만 쓰는사람같음 크로스체크 요망
from pyspark.sql.functions import col, when, isnan, count

# 1. _2순위신용체크구분 결측치 비율 확인 (이용카드수_신용체크 기준)
df1.select(
    "이용카드수_신용체크",
    "_2순위신용체크구분"
).withColumn(
    "is_null_2nd",
    col("_2순위신용체크구분").isNull()
).groupBy("이용카드수_신용체크").agg(
    count(when(col("is_null_2nd"), True)).alias("결측치수"),
    count("*").alias("전체수")
).withColumn(
    "결측비율(%)", (col("결측치수") / col("전체수") * 100).cast("double")
).orderBy("이용카드수_신용체크").display()


In [0]:
# 1. 결측치인 행만 필터링
null_df = df1.filter(col("_2순위신용체크구분").isNull())

# 2. 결측치 집단에서 카드 수별 인원 카운트
from pyspark.sql.functions import count

null_grouped = null_df.groupBy("이용카드수_신용체크").agg(count("*").alias("결측치_수"))
null_pd = null_grouped.orderBy("이용카드수_신용체크").toPandas()

# 3. 비율 계산
null_pd["결측비율(%)"] = (null_pd["결측치_수"] / null_pd["결측치_수"].sum()) * 100

# 4. 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
bars = plt.bar(null_pd["이용카드수_신용체크"], null_pd["결측비율(%)"], color="salmon", edgecolor="black")

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f'{height:.1f}%', ha='center', va='bottom')

plt.title("2순위 신용체크구분 결측치 중 카드 보유수 분포 (%)")
plt.xlabel("이용카드수 (신용+체크)")
plt.ylabel("결측치 집단 내 비율 (%)")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [0]:
# 2. _1순위신용체크구분 결측치 비율 확인 (이용카드수_신용체크 == 0)
df1.filter(col("이용카드수_신용체크") == 0).select(
    count(when(col("_1순위신용체크구분").isNull(), True)).alias("결측치수"),
    count("*").alias("전체수")
).withColumn(
    "결측비율(%)", (col("결측치수") / col("전체수") * 100)
).display()


In [0]:
#한글
!apt-get update -qq
!apt-get install -y fonts-nanum

In [0]:
df1.filter("`이용카드수_신용체크` = 0").groupby("_2순위신용체크구분").count().orderBy("count", ascending=False).display()

#이용카드수가 0인사람중에서 _2값 기준 그룹화 후 카운드, 카드는 없는데 이러면 이용 가능 카드로 본다?

#_2순위신용체크구분	count
#null	1835518
#신용	854012
#체크	578854

#총 326만명이 이용카드수 0인상태, 이중에서 183만명(56퍼)가 결측치
# 나머지 40퍼는 신용/체크 사용내역이 없지만 2순위 구분값 존재
# 과거 사용정보 남앗다?
#보유와 사용은 다름(아래 행보면 304만명 93.3%)
#이용카드수_신용체크는 최근 3개월간 카드이용 0인데 2순위있는건 이상가능성 존재, 탐색 ㅛㅇ망 >이전사용이력 잔존? 데이터 오류?

In [0]:
df1.filter("`이용카드수_신용체크` = 0 AND `유효카드수_신용체크` > 0").count()
#이요카드수 0이면서, 유효카드수는 0보다 큰사람 찾아 즉 이용카드는 0개 유효(등록)카드는 1개 이상
#300만건 나옴 -> 카드는 발급 받고 3개월 기준 사용하지 않은거같다. 단순 미사용이 존재?
#결과값은 3048706

In [0]:
from pyspark.sql.functions import col, when

# 의심 케이스만 필터링
suspicious_df = df1.filter(
    (col("이용카드수_신용체크") == 0) &
    (col("_2순위신용체크구분").isNotNull())
)

# 관련 변수 몇 개 붙여서 확인
suspicious_df.select(
    "발급회원번호",
    "유효카드수_신용체크",
    "이용가능카드수_신용체크",
    "이용금액_R3M_신용체크",
    "_1순위카드이용건수",
    "_1순위카드이용금액",
    "최종유효년월_신용_이용",
    "최종카드발급경과월",
    "_2순위신용체크구분"
).display()
# 1순위카드이용건수가 왜 음수가나오고, 양수일경우 이용금액이 0이 나오는데 버근가? 이거 뭐임?
# 이용건수 음수는 제거 요망, 양수일때 이용금액 0이면 취소, 정정처리? 이용금액이 마이너스는 무엇인가(기간의 차이? 3개월 6개월?)근데 7월부터 12월 데이터인데?

In [0]:
df1.filter(col("_1순위카드이용건수") < 0).display()
#5443개가 왜이러지,,? 심지어 금액도 음수가있어

In [0]:
#이용건수 0 무조건 이상치인데이건..?
df1.filter(col("_1순위카드이용건수") < 0).count()


In [0]:
#이용건수는 양수 &이용금액은 0
df1.filter(
    (col("_1순위카드이용건수") > 0) &
    (col("_1순위카드이용금액") == 0)
).count()


In [0]:
#이용건수는 0 이용금액은 양수
df1.filter(
    (col("_1순위카드이용건수") == 0) &
    (col("_1순위카드이용금액") > 0)
).count()


In [0]:
df1.filter(col("_1순위카드이용건수") < 0).limit(20).display()
#20개만 보는중, 이용금액 신용이랑 체크가 음수긴 한데 환불 이런거때문에 설명은됨
#도대체 어케 해야 음수나옴? 나머진 대부분 정상적 기입인걸 보면 오류말고는 답도 없느데
#life_stage랑 펀드같은거 묶어서 분석할떄, 
#방안_1._1순위 카드 이용건수를 뺴고 금액만 분석한다,,?
#방안_2. 이용금액적인걸 04.카드승인매출에서 분석? 근데 01 회원정보에서 음수가 나오면 03카드매출에서도 당연히 음수아님? >>일단 03에서 이용건수,이용금액 문의
###### 세빈님 말대로 생각해보면 취소 환불나오면 이용건수도 -로 된다 아님? 어 근데 그러면 0이어야하는거아닌가?

In [0]:
from pyspark.sql.functions import col, count, avg, sum, when

# 최종카드발급일자 결측치인 사용자 필터링
df1_issue_null = df1.filter(col("최종카드발급일자").isNull())

# 연령, 성별 기준 그룹화 및 요약 통계 생성
df_grouped_issue = df1_issue_null.groupBy("연령", "남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg(col("_1순위카드이용금액")).alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
    sum(
        when(col("최종유효년월_신용_이용").isNull(), 1).otherwise(0)
    ).alias("이용_컬럼_Null_건수")  # 추가 분석 포인트
)

# 결과 정렬 및 출력
display(df_grouped_issue.orderBy("연령"))


In [0]:
from pyspark.sql.functions import col, count, avg, sum, when
import matplotlib.pyplot as plt

# ✅ 1. 최종카드발급일자 결측 사용자 필터링
df1_issue_null = df1.filter(col("최종카드발급일자").isNull())

# ✅ 2. 그룹화 및 요약 통계
df_grouped_issue = df1_issue_null.groupBy("연령", "남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg(col("_1순위카드이용금액")).alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
    sum(when(col("최종유효년월_신용_이용").isNull(), 1).otherwise(0)).alias("이용_컬럼_Null_건수")
)

# ✅ 3. toPandas 변환
pdf = df_grouped_issue.toPandas()

# ✅ 4. 성별 라벨 정의
labels = {1: "남성", 2: "여성"}
colors = {1: "blue", 2: "red"}

# ✅ 5. 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharey=False)  # << sharey=False 포인트
fig.suptitle("최종카드발급일자 결측 사용자 분석 (연령 × 성별)", fontsize=16)

metrics = [
    ("유효카드수_평균", "유효카드수 평균", "카드수"),
    ("1순위카드이용금액_평균", "1순위카드 이용금액 평균", "평균"),
    ("이용금액_0원_건수", "이용금액 0원 건수", "건수"),
    ("이용_컬럼_Null_건수", "이용 컬럼 Null 건수", "건수")
]

for ax, (col_name, title, ylabel) in zip(axes.flatten(), metrics):
    for gender in [1, 2]:
        data = pdf[pdf["남녀구분코드"] == gender].sort_values("연령")
        ax.plot(data["연령"], data[col_name], marker='o', label=labels[gender], color=colors[gender])
    
    ax.set_title(title)
    ax.set_xlabel("연령")
    ax.set_ylabel(ylabel)
    ax.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [0]:
from pyspark.sql.functions import col

# 조건: 발급일자가 Null인데 유효카드 수가 1 이상
df_cardholders_with_null_issue_date = df1.filter(
    col("최종카드발급일자").isNull() & 
    (col("유효카드수_신용체크") >= 1)
)

# 샘플 10개만 출력
df_cardholders_with_null_issue_date.select(
    "연령", "남녀구분코드", "유효카드수_신용체크", 
    "_1순위카드이용금액", "최종유효년월_신용_이용", "최종카드발급일자"
).show(10, truncate=False)


In [0]:
from pyspark.sql.functions import col

# 조건에 맞는 사람 수 세기
count_with_null_issue_and_card = df1.filter(
    col("최종카드발급일자").isNull() &
    (col("유효카드수_신용체크") >= 1)
).count()

print(f"최종카드발급일자가 Null인데 유효카드수를 1개 이상 가진 사람 수: {count_with_null_issue_and_card}")


In [0]:
from pyspark.sql.functions import col, count

df1.filter(
    col("최종카드발급일자").isNull() &
    (col("유효카드수_신용체크") >= 1)
).groupBy("연령", "남녀구분코드").agg(
    count("*").alias("인원수")
).orderBy("연령", "남녀구분코드").show()


In [0]:
df1.filter(col("최종카드발급일자") == "0").count()


In [0]:
df1.filter(col("최종카드발급일자") == 0).count()
df1.filter(col("최종카드발급일자") == "Null").count()



In [0]:
df1.select("최종카드발급일자").distinct().show(truncate=False)


In [0]:
from pyspark.sql.functions import count

df1.groupBy("최종카드발급일자").agg(count("*").alias("건수")).orderBy("건수", ascending=False).show(truncate=False)


In [0]:
df1.groupBy("최종카드발급일자") \
   .agg(count("*").alias("건수")) \
   .orderBy(col("최종카드발급일자").desc()) \
   .show(truncate=False)


In [0]:
# 위에건 유니크값 몇개인지
# 이건 유니크값 수

df1.select("최종카드발급일자").distinct().count()


In [0]:
from pyspark.sql.functions import col, rand
import random

# 조건: 최종카드발급일자가 Null이고, 유효카드수(신용/체크)가 1 이상
df_null_issue = df1.filter(
    (col("최종카드발급일자").isNull()) &
    (col("유효카드수_신용체크") >= 1)
)

# 무작위 10개 추출
df_sample = df_null_issue.orderBy(rand()).limit(10)

# 전체 열 보기
display(df_sample)


In [0]:
df1.filter(col("가입통신회사코드") == Null)

In [0]:
from pyspark.sql.functions import avg, count

df_grouped = df1_test_phone.groupBy("연령").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg("_1순위카드이용금액").alias("1순위카드이용금액_평균")
)

display(df_grouped.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, lit

df1_test_phone = df1.filter(col("최종유효년월_신용_이용").isNull())

display(df1_test_phone.limit(3))

In [0]:
df_grouped = df1_test_phone.groupBy("연령","남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg("_1순위카드이용금액").alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
)

display(df_grouped.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, lit

df1_minus = df1.filter(col("_1순위카드이용건수") < 0)

display(df1_minus.groupBy("기준년월").count())

In [0]:
df_grouped = df1_minus.groupBy("연령","남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg("_1순위카드이용금액").alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
)

display(df_grouped.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, lit

df1_plus = df1.filter(col("_1순위카드이용건수") > 0)

df_grouped = df1_plus.groupBy("연령","남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg("_1순위카드이용금액").alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
)

display(df_grouped.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, count, sum, when, round

df_invalid_ratio = df1.groupBy("연령").agg(
    count(when(col("_1순위카드이용건수").isNotNull(), True)).alias("전체_건수"),
    sum(when(col("_1순위카드이용건수") < 0, 1).otherwise(0)).alias("음수_건수")
).withColumn(
    "음수_비율(%)", round((col("음수_건수") / col("전체_건수")) * 100, 2)
)

display(df_invalid_ratio.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, lit

df1_job = df1.filter(col("직장시도명").isNull())

df_grouped = df1_job.groupBy("연령","남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg("_1순위카드이용금액").alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
)

display(df_grouped.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, lit

df1_job = df1.filter(col("_1순위신용체크구분").isNull())

df_grouped = df1_job.groupBy("연령","남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg("_1순위카드이용금액").alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
)

display(df_grouped.orderBy("연령"))

In [0]:
from pyspark.sql.functions import col, count, avg, sum, when

df1_job = df1.filter(col("최종유효년월_신용_이용가능").isNull())

df_grouped = df1_job.groupBy("연령", "남녀구분코드").agg(
    count("*").alias("이용자수"),
    avg("유효카드수_신용체크").alias("유효카드수_평균"),
    avg(col("_1순위카드이용금액")).alias("1순위카드이용금액_평균"),
    sum(when(col("_1순위카드이용금액") == 0, 1).otherwise(0)).alias("이용금액_0원_건수"),
    sum(
        when(
            col("최종유효년월_신용_이용").isNull(),
            1
        ).otherwise(0)
    ).alias("이용_컬럼_Null_건수")  # 추가: 이용컬럼도 null인 경우
)

display(df_grouped.orderBy("연령"))

In [0]:
display(df1_job.limit(4))

##결측치 대체 시작
결측치 제거 및 날짜형 변수 범주형으로 바꾼 df는 df1.1로 해서 수정예정


날짜형 변수는 월별로할지 연도별로할지 분기별로할지 반기별로할지 고민중

최종유효년월_신용_이용,최종유효년월_신용_이용가능



| 컬럼명 | 결측치 개수 | 결측치 비율 | 결측치 처리 방식 |
|---|---|---|-----|
| _2순위신용체크구분 | 7,189,389 | 39.94% | "없음"으로 대체(현금사용 or 1순위카드만사용) |
| 최종유효년월_신용_이용 | 4,031,543 | 22.40% | "없음"으로대체(현금유저), 날짜형식 범주형으로 변환 요망 |
| 가입통신회사코드 | 2,910,604 | 16.17% | "알뜰폰"으로 대체 |
| 직장시도명 | 1,829,715 | 10.17% | "비경제"대체 |
| 최종유효년월_신용_이용가능 | 1,590,648 | 8.84% | "없음"으로대체(현금유저),날짜형식 범주형으로 변환 요망 |
| 최종카드발급일자 | 319,362 | 1.77% |  행 삭제 |
| _1순위신용체크구분 | 211,517 | 1.18% | "현금"으로대체(현금유저) |

In [0]:
#원본 남기고 df1_1
df1_1 = df1
print(f"df1_1에 복사된 행 수: {df1_1.count()}")

In [0]:
# _2순위신용체크구분 컬럼의 결측치를 "현금"으로 대체
from pyspark.sql.functions import col, when


df1_1 = df1_1.withColumn(
    "_2순위신용체크구분",
    when(col("_2순위신용체크구분").isNull(), "현금").otherwise(col("_2순위신용체크구분"))
)

print("\n'_2순위신용체크구분' 컬럼 처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, 결측치 개수를 출력합니다.
df1_1.select("_2순위신용체크구분").show(5, truncate=False)

# 처리 후 결측치 개수 확인
from pyspark.sql.functions import count
null_count_after = df1_1.filter(col('_2순위신용체크구분').isNull()).count()
print(f"처리 후 '_2순위신용체크구분' 결측치 개수: {null_count_after}")

# 컬럼의 고유값 확인 (선택 사항)
df1_1.select("_2순위신용체크구분").distinct().show(truncate=False)

In [0]:
# 최종유효년월_신용_이용 컬럼의 결측치를 "없음"으로 대체 ++ 범주형으로 다 수정예정
from pyspark.sql.functions import col, when


df1_1 = df1_1.withColumn(
    "최종유효년월_신용_이용",
    when(col("최종유효년월_신용_이용").isNull(), "없음").otherwise(col("최종유효년월_신용_이용"))
)

print("\n'최종유효년월_신용_이용' 컬럼 처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, 결측치 개수를 출력합니다.
df1_1.select("최종유효년월_신용_이용").show(5, truncate=False)

# 처리 후 결측치 개수 확인
from pyspark.sql.functions import count
null_count_after = df1_1.filter(col('최종유효년월_신용_이용').isNull()).count()
print(f"처리 후 '최종유효년월_신용_이용' 결측치 개수: {null_count_after}")

# 컬럼의 고유값 확인 (선택 사항)
# df1_1.select("최종유효년월_신용_이용").distinct().show(20, truncate=False) # 고유값이 많을 수 있으니 상위 20개만

In [0]:
#최종유효년울~이용 잘된건지 확인
df1_1.select("최종유효년월_신용_이용").show(10, truncate=False)

In [0]:
# 전체 행 확인
from pyspark.sql.functions import col

print("\n'최종유효년월_신용_이용' 컬럼의 고유 값 및 개수:")
df1_1.groupBy("최종유효년월_신용_이용").count().orderBy("count", ascending=False).show(50, truncate=False)

In [0]:
# 가입통신회사코드 컬럼의 결측치를 "알뜰폰"으로 대체
from pyspark.sql.functions import col, when


df1_1 = df1_1.withColumn(
    "가입통신회사코드",
    when(col("가입통신회사코드").isNull(), "알뜰폰").otherwise(col("가입통신회사코드"))
)

print("\n'가입통신회사코드' 컬럼 처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, 결측치 개수를 출력합니다.
df1_1.select("가입통신회사코드").show(5, truncate=False)

# 처리 후 결측치 개수 확인
from pyspark.sql.functions import count
null_count_after = df1_1.filter(col('가입통신회사코드').isNull()).count()
print(f"처리 후 '가입통신회사코드' 결측치 개수: {null_count_after}")

# 컬럼의 고유값 및 개수 확인 (선택 사항)
df1_1.groupBy("가입통신회사코드").count().orderBy("count", ascending=False).show(20, truncate=False)

In [0]:
# "etc" 값이 포함된 행의 개수 확인 :설명서엔있는데 여긴없어
from pyspark.sql.functions import col


etc_count = df1_1.filter(col("가입통신회사코드") == "etc").count()

if etc_count > 0:
    print(f"'가입통신회사코드' 컬럼에 'etc' 값이 {etc_count}개 존재합니다.")
    # 존재한다면, 해당 값을 가진 데이터의 일부를 확인해볼 수도 있습니다.
    df1_1.filter(col("가입통신회사코드") == "etc").select("가입통신회사코드").show(5, truncate=False)
else:
    print("'가입통신회사코드' 컬럼에 'etc' 값이 존재하지 않습니다.")

In [0]:
# 직장시도명 컬럼의 결측치를 "비경제"로 대체
from pyspark.sql.functions import col, when


df1_1 = df1_1.withColumn(
    "직장시도명",
    when(col("직장시도명").isNull(), "비경제").otherwise(col("직장시도명"))
)

print("\n'직장시도명' 컬럼 처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, 결측치 개수를 출력합니다.
df1_1.select("직장시도명").show(5, truncate=False)

# 처리 후 결측치 개수 확인
from pyspark.sql.functions import count
null_count_after = df1_1.filter(col('직장시도명').isNull()).count()
print(f"처리 후 '직장시도명' 결측치 개수: {null_count_after}")

# 컬럼의 고유값 및 개수 확인 (선택 사항)
df1_1.groupBy("직장시도명").count().orderBy("count", ascending=False).show(20, truncate=False)

In [0]:
# 최종유효년월_신용_이용가능 컬럼의 결측치를 "없음"으로 대체
from pyspark.sql.functions import col, when


df1_1 = df1_1.withColumn(
    "최종유효년월_신용_이용가능",
    when(col("최종유효년월_신용_이용가능").isNull(), "없음").otherwise(col("최종유효년월_신용_이용가능"))
)

print("\n'최종유효년월_신용_이용가능' 컬럼 처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, 결측치 개수를 출력합니다.
df1_1.select("최종유효년월_신용_이용가능").show(5, truncate=False)

# 처리 후 결측치 개수 확인
from pyspark.sql.functions import count
null_count_after = df1_1.filter(col('최종유효년월_신용_이용가능').isNull()).count()
print(f"처리 후 '최종유효년월_신용_이용가능' 결측치 개수: {null_count_after}")

# 컬럼의 고유값 및 개수 확인 (선택 사항)
# df1_1.groupBy("최종유효년월_신용_이용가능").count().orderBy("count", ascending=False).show(20, truncate=False)

In [0]:
#아까처럼 확인
from pyspark.sql.functions import col

print("\n'최종유효년월_신용_이용가능' 컬럼의 고유 값 및 개수:")
df1_1.groupBy("최종유효년월_신용_이용가능").count().orderBy("count", ascending=False).show(50, truncate=False)

In [0]:
# 최종카드발급일자 컬럼에 null 값이 있는 행 삭제 전 행 수 확인 1800만에서 1768만개로 32만개정도 삭제
initial_rows = df1_1.count()
print(f"행 삭제 전 df1_1의 총 행 수: {initial_rows}")

# 최종카드발급일자 컬럼에 null 값이 있는 행 삭제
# na.drop(subset=["컬럼명"])은 해당 컬럼에 null이 있는 행을 모두 삭제합니다.
df1_1 = df1_1.na.drop(subset=["최종카드발급일자"])

print("\n'최종카드발급일자' 컬럼 처리 후 (행 삭제):")
print(f"삭제 후 df1_1의 총 행 수: {df1_1.count()}")

# 삭제 후 최종카드발급일자 컬럼의 결측치 개수 확인
from pyspark.sql.functions import col, count
null_count_after = df1_1.filter(col('최종카드발급일자').isNull()).count()
print(f"처리 후 '최종카드발급일자' 결측치 개수: {null_count_after}")

# 삭제된 행이 있는지 확인하기 위해 일부 데이터 출력 (선택 사항)
# df1_1.select("최종카드발급일자").show(5, truncate=False)

In [0]:
# _1순위신용체크구분 컬럼의 결측치를 "현금"으로 대체
from pyspark.sql.functions import col, when


df1_1 = df1_1.withColumn(
    "_1순위신용체크구분",
    when(col("_1순위신용체크구분").isNull(), "현금").otherwise(col("_1순위신용체크구분"))
)

print("\n'_1순위신용체크구분' 컬럼 처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, 결측치 개수를 출력합니다.
df1_1.select("_1순위신용체크구분").show(5, truncate=False)

# 처리 후 결측치 개수 확인
from pyspark.sql.functions import count
null_count_after = df1_1.filter(col('_1순위신용체크구분').isNull()).count()
print(f"처리 후 '_1순위신용체크구분' 결측치 개수: {null_count_after}")

# 컬럼의 고유값 및 개수 확인 (선택 사항)
df1_1.groupBy("_1순위신용체크구분").count().orderBy("count", ascending=False).show(20, truncate=False)

In [0]:
# _2순위신용체크구분 컬럼의 "현금" 값을 "없음"으로 대체
# '현금'이 아닌 다른 값들(체크, 신용 등)은 그대로 유지됩니다.
from pyspark.sql.functions import col, when

print("'_2순위신용체크구분' 컬럼: '현금' 값을 '없음'으로 재대체 시작합니다.")


df1_1 = df1_1.withColumn(
    "_2순위신용체크구분",
    when(col("_2순위신용체크구분") == "현금", "없음")
    .otherwise(col("_2순위신용체크구분"))
)

print("\n'_2순위신용체크구분' 컬럼 재처리 후:")
# 변경 사항 확인을 위해 일부 데이터와 해당 컬럼의 고유값, '없음' 개수를 출력합니다.
df1_1.select("_2순위신용체크구분").show(5, truncate=False)

# 처리 후 '현금' 값 개수 확인 (0이어야 정상)
current_cash_count = df1_1.filter(col('_2순위신용체크구분') == "현금").count()
print(f"처리 후 '_2순위신용체크구분' 컬럼의 '현금' 값 개수: {current_cash_count}")

# '없음' 값 개수 확인
new_none_count = df1_1.filter(col('_2순위신용체크구분') == "없음").count()
print(f"처리 후 '_2순위신용체크구분' 컬럼의 '없음' 값 개수: {new_none_count}")

# 컬럼의 최종 고유값 확인
df1_1.select("_2순위신용체크구분").distinct().show(20, truncate=False)

In [0]:
# 최종유효년월_신용_이용 컬럼의 연도별 개수 확인
from pyspark.sql.functions import col, substring, expr


print("\n'최종유효년월_신용_이용' 컬럼 - 연도별 데이터 개수:")
df1_1.withColumn(
    "이용_연도",
    # '없음' 값은 연도 추출 대상이 아니므로 조건부로 처리
    when(col("최종유효년월_신용_이용") == "없음", "없음")
    .otherwise(substring(col("최종유효년월_신용_이용"), 1, 4)) # 앞 4자리 추출
).groupBy("이용_연도").count().orderBy("이용_연도").show(50, truncate=False)

# 최종유효년월_신용_이용가능 컬럼의 연도별 개수 확인
print("\n'최종유효년월_신용_이용가능' 컬럼 - 연도별 데이터 개수:")
df1_1.withColumn(
    "이용가능_연도",
    # '없음' 값은 연도 추출 대상이 아니므로 조건부로 처리
    when(col("최종유효년월_신용_이용가능") == "없음", "없음")
    .otherwise(substring(col("최종유효년월_신용_이용가능"), 1, 4)) # 앞 4자리 추출
).groupBy("이용가능_연도").count().orderBy("이용가능_연도").show(50, truncate=False)

# 참고: 원본 데이터가 날짜 형식으로 완벽하지 않을 수 있으므로,
# 실제 날짜 함수 (to_date, year 등)를 사용하기 전 문자열 형태를 확인하는 것이 좋습니다.
# 만약 '없음' 외에 8자리 숫자가 아닌 값이 있다면, substring이 오류를 내지 않고
# 빈 문자열을 반환할 수 있으므로 주의 깊게 살펴봐야 합니다.

In [0]:
#  최종유효년월_신용_이용, 가능 컬럼을 연도 단위 범주형으로 변환

from pyspark.sql.functions import col, substring, when

print("날짜 컬럼을 연도 단위 범주형으로 변환 시작합니다.")

# 1. 최종유효년월_신용_이용 컬럼을 연도 단위 범주형으로 변환
df1_1 = df1_1.withColumn(
    "최종유효년월_신용_이용_연도",
    when(col("최종유효년월_신용_이용") == "없음", "없음")  # '없음'은 그대로 '없음'으로 유지
    .otherwise(substring(col("최종유효년월_신용_이용"), 1, 4)) # 날짜에서 연도 4자리 추출
)

# 2. 최종유효년월_신용_이용가능 컬럼을 연도 단위 범주형으로 변환
df1_1 = df1_1.withColumn(
    "최종유효년월_신용_이용가능_연도",
    when(col("최종유효년월_신용_이용가능") == "없음", "없음") # '없음'은 그대로 '없음'으로 유지
    .otherwise(substring(col("최종유효년월_신용_이용가능"), 1, 4)) # 날짜에서 연도 4자리 추출
)

print("\n'최종유효년월_신용_이용_연도' 컬럼 생성 후:")
df1_1.select("최종유효년월_신용_이용", "최종유효년월_신용_이용_연도").show(5, truncate=False)
df1_1.groupBy("최종유효년월_신용_이용_연도").count().orderBy("최종유효년월_신용_이용_연도").show(20, truncate=False)

print("\n'최종유효년월_신용_이용가능_연도' 컬럼 생성 후:")
df1_1.select("최종유효년월_신용_이용가능", "최종유효년월_신용_이용가능_연도").show(5, truncate=False)
df1_1.groupBy("최종유효년월_신용_이용가능_연도").count().orderBy("최종유효년월_신용_이용가능_연도").show(20, truncate=False)

print("\n연도 단위 범주형 변환이 완료되었습니다.")

In [0]:
display(df1_1.limit(4))

### card_member_info_processed
이 거로 저장함 
경로는
dbfs:/user/hive/warehouse/database_pjt.db/card_member_info_processed

In [0]:
#누군가가 쏘아올린 작은 공 ㅠㅠ
# 1. 저장할 데이터베이스와 테이블 이름 설정
database_name = "database_pjt"  # 데이터베이스 이름
table_name = "card_member_info_processed" # 저장 테이블 아 이거 df1_1로할껄 그랫나?
print(f"데이터베이스 '{database_name}'에 '{table_name}' 테이블 저장을 시도합니다.")

# 2. 데이터베이스 생성 (이미 존재하면 건너뜀)
# 이 스키마는 hive_metastore에 생성됩니다.
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
print(f"데이터베이스 '{database_name}' 확인 또는 생성 완료.")

# 3. DataFrame을 Hive Metastore 테이블로 저장 (Delta Lake 형식)
# "overwrite" 모드를 사용하여 기존 테이블이 있다면 덮어씁니다.
df1_1.write \
    .format("delta") \
    .mode("overwrite") \
    .option("path", f"dbfs:/user/hive/warehouse/{database_name}.db/{table_name}") \
    .saveAsTable(f"{database_name}.{table_name}")

print(f"\n'{database_name}.{table_name}' 테이블이 hive_metastore에 성공적으로 저장되었습니다.")
print("Databricks UI에서 **Catalog -> hive_metastore -> database_pjt** 에서 확인하실 수 있습니다.")

# 4. 저장된 테이블 확인 (선택 사항)
print(f"\n저장된 테이블의 첫 5개 행:")
spark.sql(f"SELECT * FROM {database_name}.{table_name} LIMIT 5").show()

print(f"\n저장된 테이블의 스키마:")
spark.sql(f"DESCRIBE {database_name}.{table_name}").show()

In [0]:
# SparkSession이 이미 생성되어 있어야 합니다.
df1_1 = spark.read.format("delta").load("dbfs:/user/hive/warehouse/database_pjt.db/card_member_info_processed")

# 데이터 확인
print(f"df1_1 행 수: {df1_1.count()}")
df1_1.printSchema()
df1_1.show(5, truncate=False)


## 이상치 파악

In [0]:
#수치형 
numerical_cols = [
    '입회경과개월수_신용', '탈회횟수_누적', '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

In [0]:
#수치형컬럼 타입 변환 실수형으로 (double type)
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

# 수치형 변수 목록 (다시 확인)
numerical_cols_to_convert = [
    '입회경과개월수_신용', '탈회횟수_누적', '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

print("수치형 컬럼들의 데이터 타입 변환을 시작합니다 (String -> DoubleType).")

for column_name in numerical_cols_to_convert:
    # 컬럼이 존재하는지 확인
    if column_name in df1_1.columns:
        # try-except 블록으로 변환 중 발생할 수 있는 오류 방지
        try:
            df1_1 = df1_1.withColumn(column_name, col(column_name).cast(DoubleType()))
            print(f"  컬럼 '{column_name}'을 DoubleType으로 변환 완료.")
        except Exception as e:
            print(f"  경고: 컬럼 '{column_name}' 변환 중 오류 발생: {e}. 해당 컬럼은 변환되지 않았습니다.")
    else:
        print(f"  경고: 컬럼 '{column_name}'이(가) DataFrame에 존재하지 않습니다. 스키마를 확인해주세요.")

print("\n데이터 타입 변환 후 df1_1의 스키마:")
df1_1.printSchema()

# 변환된 컬럼들의 요약 통계 다시 확인 (min, max 등이 숫자로 나와야 합니다)
print("\n변환된 수치형 컬럼들의 요약 통계 (일부 예시):")
df1_1.select('이용금액_R3M_신용', '카드신청건수', '탈회횟수_누적').summary().show()

In [0]:
#이상치값확인
from pyspark.sql.functions import col, percentile_approx, lit, when
from pyspark.sql.types import DoubleType, IntegerType, LongType, FloatType # 여기를 수정했습니다!

# 수치형 변수 목록
numerical_cols = [
    '입회경과개월수_신용', '탈회횟수_누적', '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

print("각 수치형 컬럼의 이상치 값 확인을 시작합니다.")

for col_name in numerical_cols:
    if col_name in df1_1.columns:
        print(f"\n--- 컬럼: {col_name} ---")

        # 해당 컬럼의 데이터 타입이 수치형인지 다시 한번 확인
        # 수정된 임포트로 인해 이제 LongType 등도 제대로 인식됩니다.
        if not isinstance(df1_1.schema[col_name].dataType, (DoubleType, IntegerType, LongType, FloatType)):
            print(f"  경고: '{col_name}' 컬럼은 수치형이 아닙니다 (현재 타입: {df1_1.schema[col_name].dataType}). 이상치 확인을 건너뜜.")
            continue

        # Q1, Q3, IQR 계산
        # approxQuantile은 정확도 0.01 (1%) 오차 허용
        quantiles = df1_1.approxQuantile(col_name, [0.25, 0.75], 0.01)
        Q1 = quantiles[0]
        Q3 = quantiles[1]
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        print(f"  Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
        print(f"  하한 경계 (Lower Bound): {lower_bound}")
        print(f"  상한 경계 (Upper Bound): {upper_bound}")

        # 이상치 필터링
        outliers_df = df1_1.filter(
            (col(col_name) < lower_bound) | (col(col_name) > upper_bound)
        )

        outlier_count = outliers_df.count()
        total_count = df1_1.count()

        if outlier_count > 0:
            outlier_percentage = (outlier_count / total_count) * 100
            print(f"  이상치 개수: {outlier_count}개 ({outlier_percentage:.2f}%)")
            print("  --- 이상치 데이터 예시 (최대 10개) ---")
            outliers_df.select(col_name).limit(10).show(truncate=False)
        else:
            print("  이상치가 발견되지 않았습니다.")
    else:
        print(f"경고: 컬럼 '{col_name}'이(가) DataFrame에 존재하지 않습니다. 스키마를 확인해주세요.")

print("\n모든 수치형 컬럼에 대한 이상치 값 확인이 완료되었습니다.")

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Spark DataFrame에서 필요한 컬럼만 선택하여 Pandas로 변환
numerical_cols_in_df = [col for col in numerical_cols if col in df1_1.columns]

# Spark → Pandas로 수치형 컬럼만 추출
df_pd = df1_1.select(numerical_cols_in_df).toPandas()

# 박스플롯 스타일 설정
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")

# 박스플롯 그리기 (컬럼별 색상 지정)
palette = sns.color_palette("Set3", len(numerical_cols_in_df))  # 색상 다양화

sns.boxplot(data=df_pd[numerical_cols_in_df], palette=palette)

# 라벨 회전 및 레이아웃 조정
plt.xticks(rotation=45, ha='right')
plt.title("수치형 컬럼별 이상치 시각화 (Boxplot)", fontsize=16)
plt.tight_layout()
plt.show()


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

columns_to_plot = [
    '입회경과개월수_신용', '탈회횟수_누적',  
    '_1순위카드이용건수', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]


# 필요한 컬럼만 선택해서 Pandas DataFrame으로 변환 (샘플링 권장)
pdf1 = df1_1.select(columns_to_plot).sample(fraction=0.1, seed=42).toPandas()

for col in columns_to_plot:
    pdf1[col] = pd.to_numeric(pdf1[col], errors='coerce')


plt.figure(figsize=(15, 8))
sns.boxplot(data=pdf1, orient='h')
plt.show()


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

columns_to_plot = [
     '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_2순위카드이용금액','_1순위카드이용건수', '_2순위카드이용건수'
]


# 필요한 컬럼만 선택해서 Pandas DataFrame으로 변환 (샘플링 권장)
pdf1 = df1_1.select(columns_to_plot).sample(fraction=0.1, seed=42).toPandas()

for col in columns_to_plot:
    pdf1[col] = pd.to_numeric(pdf1[col], errors='coerce')


plt.figure(figsize=(15, 8))
sns.boxplot(data=pdf1, orient='h')
plt.show()


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# 시각화용 임시 컬럼명 매핑
columns_to_plot = [
    '입회경과개월수_신용', '탈회횟수_누적', '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

# 샘플링 및 Pandas 변환
pdf1 = df1_1.select(columns_to_plot).sample(fraction=0.1, seed=42).toPandas()

# 숫자형 변환 및 로그 변환
for col in columns_to_plot:
    pdf1[col] = pd.to_numeric(pdf1[col], errors='coerce')
    pdf1[col] = np.log1p(pdf1[col])

# 컬럼명 변경 (시각화 목적)
rename_dict = dict(zip(columns_to_plot, english_names))

# 시각화
plt.figure(figsize=(15, 8))
sns.boxplot(data=pdf1, orient='h', palette="Set3")
plt.title("Boxplot of log-transformed variables", fontsize=14)
plt.xlabel("log(1 + x)")
plt.tight_layout()
plt.show()


# 📊 이상치 분석 결과 및 처리 계획

## 1. 이상치 분석 결과 요약

각 수치형 컬럼에 대해 IQR(Interquartile Range) 기반으로 이상치를 분석한 결과는 다음과 같습니다.

### 1.1. `입회경과개월수_신용`
* **이상치 개수**: 993,810개 (5.62%)
* **상한 경계**: 232.0개월
* **하한 경계**: -112.0개월 (음수 경계는 데이터 분포의 치우침을 나타내며, 실제 경과 개월 수가 음수일 수는 없음)
* **해석**: 232개월(약 19년)을 초과하는 매우 장기 회원들이 이상치로 식별되었습니다. 이는 장기 회원의 특성을 반영하거나 특정 시점 이후의 회원 유입 패턴과 관련될 수 있습니다.

### 1.2. `탈회횟수_누적`
* **이상치 없음**
* **해석**: 대부분의 데이터가 0, 1, 2와 같은 낮은 정수 값으로 분포하여 IQR 기준으로 이상치가 발견되지 않았습니다.

### 1.3. `이용금액_R3M_신용`
* **이상치 개수**: 1,464,257개 (8.28%)
* **상한 경계**: 4,926,760.0원
* **하한 경계**: -2,956,056.0원 (이용금액이 음수인 경우는 환불과 같은 특수 케이스로 판단됨)
* **해석**: 500만원에 가까운 금액을 초과하는 고액 이용자들이 이상치로 식별되었습니다. 이는 카드 이용 패턴에서 중요한 세그먼트가 될 수 있습니다.

### 1.4. `이용금액_R3M_체크`
* **이상치 개수**: 2,811,447개 (15.90%)
* **상한/하한 경계**: 모두 0.0원
* **해석**: 이 컬럼의 대부분 데이터가 0원이므로, 0이 아닌 모든 값(즉, 체크카드 이용 금액이 있는 경우)이 IQR 기준으로는 이상치로 판단되었습니다. 이는 데이터의 자연스러운 분포 특성으로, 이상치라기보다는 특정 그룹을 나타내는 값으로 이해할 수 있습니다.

### 1.5. `_1순위카드이용금액`, `_1순위카드이용건수`, `_2순위카드이용금액`, `_2순위카드이용건수`
* **이상치 비율**: 모두 상당한 비율(5% ~ 15%)의 이상치를 포함하고 있습니다.
* **해석**: 이 이용 금액 및 건수 관련 컬럼들은 데이터가 0에 가까운 값들에 집중되어 있고, 소수의 고액/고빈도 이용자가 존재하기 때문에 이상치가 많이 잡히는 경향을 보입니다.

### 1.6. `청구금액_기본연회비_B0M`, `청구금액_제휴연회비_B0M`
* **이상치 비율**: 매우 낮은 비율(0.04% ~ 0.13%)의 이상치만 존재합니다.
* **해석**: Q1, Q3, IQR이 모두 0인 것으로 보아, 대부분의 고객이 연회비가 0원이며, 연회비가 부과된 경우에만 이상치로 잡혔습니다.

### 1.7. `카드신청건수`
* **이상치 개수**: 1,564,982개 (8.85%)
* **상한/하한 경계**: 모두 0.0
* **해석**: `이용금액_R3M_체크`와 유사하게, 대부분의 고객이 카드 신청 건수가 0이며, 한 건이라도 신청한 경우(1.0)가 이상치로 판단됩니다.

### 1.8. `최종카드발급경과월`
* **이상치 개수**: 6,153개 (0.03%)
* **상한 경계**: 59.5개월
* **해석**: 59.5개월(약 5년)을 초과하는 경과월을 가진 데이터는 소량만 이상치로 식별되었습니다.

---

## 2. 다음 단계: 이상치 처리 방식 확정 및 적용

위 분석 결과를 바탕으로, 각 수치형 컬럼의 이상치를 처리하는 방식을 다음과 같이 확정합니다.

### 2.1. `탈회횟수_누적`
* **처리 계획**: 이상치가 발견되지 않았으므로, **별도의 이상치 처리를 적용하지 않습니다.**

### 2.2. `청구금액_기본연회비_B0M`, `청구금액_제휴연회비_B0M`, `카드신청건수`, `이용금액_R3M_체크`
* **컬럼 특성**: 이 컬럼들은 Q1, Q3, IQR이 0인 경우가 많아, 0이 아닌 모든 값이 IQR 기준으로는 이상치로 잡히는 특성을 가지고 있습니다. 이 경우 IQR 기반 **캡핑(Capping)을 적용하면 0이 아닌 값들이 상한 경계값으로 변경**될 가능성이 높습니다.
* **처리 계획**: 현재 단계에서는 일단 **모든 수치형 컬럼에 대해 일관되게 IQR 기반 캡핑 로직을 적용**하겠습니다. 이후 모델 학습 과정에서 이 컬럼들의 분포가 여전히 문제가 되거나 모델 성능에 부정적인 영향을 미친다면, **로그 변환(Log Transformation)**이나 **이진 변수화(Binary Encoding)** (예: 이용/신청 여부 0 또는 1)와 같은 추가적인 피처 엔지니어링을 고려할 수 있습니다.

### 2.3. 나머지 수치형 컬럼
* **컬럼 특성**: 이 컬럼들은 비교적 높은 비율의 이상치를 포함하고 있으며, 이 이상치들은 실제 고액 이용자, 장기 회원 등 중요한 정보를 담고 있을 수 있습니다.
* **처리 계획**: **데이터 손실을 최소화**하면서 이상치의 극단적인 영향을 줄이기 위해, **IQR 기반의 캡핑(Capping) 방식**을 적용하겠습니다. 즉, 이상치로 판단되는 값들을 해당 컬럼의 계산된 상한(Upper Bound) 또는 하한(Lower Bound) 값으로 대체합니다.

---

## 3. 최종 결정: 모든 수치형 컬럼에 IQR 기반 캡핑 적용

**`탈회횟수_누적` 컬럼을 포함하여 식별된 모든 수치형 컬럼에 대해 이전에 준비했던 IQR 기반 캡핑 로직을 실행하겠습니다.** `탈회횟수_누적`과 같이 이상치가 없는 컬럼은 캡핑을 해도 값이 변하지 않을 것이며, 나머지 컬럼들은 극단적인 값들이 지정된 상한/하한으로 대체될 것입니다.



In [0]:
기본연회비_B0M
제휴연회비_B0M
상품관련면제카드수_B0M
임직원면제카드수_B0M
우수회원면제카드수_B0M
기타면제카드수_B0M
카드신청건수
최종카드발급경과월

In [0]:
할인금액_기본연회비_B0M
할인금액_제휴연회비_B0M
청구금액_기본연회비_B0M
청구금액_제휴연회비_B0M

In [0]:
display(df1_1.select('할인금액_기본연회비_B0M','할인금액_제휴연회비_B0M','청구금액_기본연회비_B0M','청구금액_제휴연회비_B0M').max())

## 이상치 제거 및 대체

연회비의 경우 0은 없음 1인 잇음으로 이진변수

그러고 컬럼 하나 더 만들어서 수치형으로 놔둠

In [0]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType

print(f"초기 데이터 행 수: {df1_1.count()}")

# 3. '최종카드발급경과월' 이상치(59.5개월 초과) 행 삭제
max_valid_month = 59.5
before_drop = df1_1.count()
df1_1 = df1_1.filter(col("최종카드발급경과월").cast(DoubleType()) <= max_valid_month)
after_drop = df1_1.count()
print(f"'최종카드발급경과월' 이상치 삭제: {before_drop - after_drop}행 삭제됨 (현재 {after_drop}행)")

# 4. 연회비 컬럼 이진 변수화 (파생 컬럼 생성, 원본은 그대로 유지)
df1_1 = df1_1.withColumn(
    "기본연회비발생여부",
    when(col("청구금액_기본연회비_B0M").cast(DoubleType()) > 0, 1).otherwise(0)
).withColumn(
    "제휴연회비발생여부",
    when(col("청구금액_제휴연회비_B0M").cast(DoubleType()) > 0, 1).otherwise(0)
)

print("연회비 관련 컬럼 이진 변수화(파생 컬럼) 생성 완료")
df1_1.select("청구금액_기본연회비_B0M", "기본연회비발생여부", "청구금액_제휴연회비_B0M", "제휴연회비발생여부").show(10, truncate=False)

# 5. 이상치 처리 제외 컬럼(원본 유지)
# - _1순위카드이용금액, _1순위카드이용건수, _2순위카드이용금액, _2순위카드이용건수, 카드신청건수
print("이상치 처리를 하지 않고 원본을 그대로 유지하는 컬럼 목록:")
print("_1순위카드이용금액, _1순위카드이용건수, _2순위카드이용금액, _2순위카드이용건수, 카드신청건수")

# 6. 분석에서 제외할 컬럼(이용금액_R3M_신용, 이용금액_R3M_체크)은 이후 분석/모델링 시 사용하지 않도록 관리

# 7. 최종 데이터 스키마 및 일부 데이터 확인
print("최종 데이터 스키마:")
df1_1.printSchema()
print("최종 데이터 일부 확인:")
df1_1.show(5, truncate=False)


In [0]:
# 확인할 컬럼 리스트
cols = [
    "청구금액_기본연회비_B0M", 
    "기본연회비발생여부", 
    "청구금액_제휴연회비_B0M", 
    "제휴연회비발생여부"
]

# 상위 10개 행 확인 (Databricks에서는 display가 가장 편리)
df1_1.select(cols).limit(10).display()


#상관관계 분석 및 컬럼축소, 변환


## 수치형 변수별 상관관계 분석 컬럼 축소

 분석 대상 수치형 변수 목록
 
입회경과개월수_신용

탈회횟수_누적

_1순위카드이용금액

_1순위카드이용건수

_2순위카드이용금액

_2순위카드이용건수

청구금액_기본연회비_B0M

청구금액_제휴연회비_B0M

카드신청건수

최종카드발급경과월

In [0]:
#상관계수확인

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import pandas as pd

# 1. 수치형 변수 리스트 정의
numerical_cols = [
    '입회경과개월수_신용', '탈회횟수_누적', '_1순위카드이용금액', '_1순위카드이용건수',
    '_2순위카드이용금액', '_2순위카드이용건수', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

# 2. VectorAssembler로 features 컬럼 생성
assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features")
df_vector = assembler.transform(df1_1.select(*numerical_cols)).select("features")

# 3. 피어슨 상관계수 행렬 계산
correlation_matrix = Correlation.corr(df_vector, "features", "pearson").head()
corr_array = correlation_matrix[0].toArray()

# 4. pandas DataFrame으로 변환하여 보기 좋게 정리
import pandas as pd
corr_df = pd.DataFrame(corr_array, index=numerical_cols, columns=numerical_cols)
display(corr_df)


| 컬럼명                  | 입회경과개월수_신용 | 탈회횟수_누적 | _1순위카드이용금액 | _1순위카드이용건수 | _2순위카드이용금액 | _2순위카드이용건수 | 청구금액_기본연회비_B0M | 청구금액_제휴연회비_B0M | 카드신청건수 | 최종카드발급경과월 |
| :---------------------- | :------------------ | :------------ | :----------------- | :----------------- | :----------------- | :----------------- | :---------------------- | :---------------------- | :----------- | :----------------- |
| **입회경과개월수_신용** | 1.000               | -0.219        | 0.108              | 0.033              | 0.070              | 0.063              | -0.025                  | 0.004                   | -0.128       | 0.248              |
| **탈회횟수_누적** | -0.219              | 1.000         | -0.022             | -0.037             | -0.015             | -0.026             | 0.007                   | -0.004                  | 0.040        | -0.108             |
| **_1순위카드이용금액** | 0.108               | -0.022        | 1.000              | 0.749              | 0.510              | 0.486              | -0.003                  | 0.026                   | 0.015        | -0.130             |
| **_1순위카드이용건수** | 0.033               | -0.037        | 0.749              | 1.000              | 0.454              | 0.534              | -0.010                  | 0.017                   | 0.014        | -0.150             |
| **_2순위카드이용금액** | 0.070               | -0.015        | 0.510              | 0.454              | 1.000              | 0.950              | 0.003                   | 0.057                   | -0.018       | -0.051             |
| **_2순위카드이용건수** | 0.063               | -0.026        | 0.486              | 0.534              | 0.950              | 1.000              | 0.001                   | 0.048                   | -0.013       | -0.055             |
| **청구금액_기본연회비_B0M** | -0.025              | 0.007         | -0.003             | -0.010             | 0.003              | 0.001              | 1.000                   | 0.177                   | 0.089        | -0.047             |
| **청구금액_제휴연회비_B0M** | 0.004               | -0.004        | 0.026              | 0.017              | 0.057              | 0.048              | 0.177                   | 1.000                   | 0.001        | -0.009             |
| **카드신청건수** | -0.128              | 0.040         | 0.015              | 0.014              | -0.018             | -0.013             | 0.089                   | 0.001                   | 1.000        | -0.311             |
| **최종카드발급경과월** | 0.248               | -0.108        | -0.130             | -0.150             | -0.051             | -0.055             | -0.047                  | -0.009                  | -0.311       | 1.000              |

In [0]:
# 상관관계 히트맵
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,8))
sns.heatmap(corr_df, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("수치형 변수들의 상관관계 히트맵")
plt.show()


### 주요 상관관계 해석

**강한 양의 상관관계**
- **2순위 금액 & 건수 (0.950)**  
  사실상 당연한 상관관계. 정보가 거의 중복되므로 둘 중 하나를 삭제해도 무방.
- **1순위 카드이용금액 & 건수 (0.749)**  
  역시 강한 상관관계. 자주 쓸수록 많이 쓴다는 의미로 해석 가능.

**중간 정도의 양의 상관관계**
- **1순위 건수 & 2순위 건수 (0.53)**  
  주력 카드와 보조 카드 간에도 금액, 건수에 긍정적 관계가 있음.
- **1순위 금액 & 2순위 금액 (0.51)**

**낮은 음의 상관관계**
- **카드신청건수 & 최종카드발급경과월 (-0.311)**  
  신청건수가 많을수록 최종경과월이 줄어드는 경향(신규 고객 특성).

**매우 낮은 상관관계**
- 대부분의 다른 컬럼 쌍은 0에 가까우며, 서로 독립적인 정보를 제공함.

**특이점**
- **탈회횟수**는 모든 변수와 상관관계가 낮아 상당히 독립적인 변수임.
- **1순위 금액/건수, 2순위 금액/건수** 4개 모두 서로 중간 이상의 관계를 보임(카드 이용 패턴이 전반적으로 비슷).

**제거할 대상?**
- **_2순위 금액 & 건수 중 하나** 삭제해도 무방
- **1순위도 동일**
- → 제거 여부는 분석 목적에 따라 결정, 놔두고 분석 시 더 잘 설명되는 변수를 선택해도 됨


## 범주형 변수 컬럼 축소

In [0]:
from pyspark.sql.functions import col, count, countDistinct, when

# 1. 범주형 변수 리스트 지정
categorical_cols = [
    '기준년월', '발급회원번호', '남녀구분코드', '연령', 'VIP등급코드', '최상위카드등급코드',
    '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용',
    '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM',
    '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드',
    '거주시도명', '직장시도명', '마케팅동의여부', 'Life_Stage'
   
]

# 2. 결측치 비율, 카테고리 개수 출력
total_rows = df1_1.count()
print(f"전체 행 수: {total_rows}\n")
print("컬럼명\t\t결측치비율\t카테고리개수")
for c in categorical_cols:
    null_cnt = df1_1.filter(col(c).isNull()).count()
    nunique = df1_1.select(countDistinct(col(c))).collect()[0][0]
    print(f"{c}\t{null_cnt/total_rows:.2%}\t{nunique}")



# 3. 카테고리 개수 기준으로 컬럼 자동 필터링(예: 20개 초과 컬럼은 제외)
max_categories = 20
useful_categorical_cols = []
for c in categorical_cols:
    nunique = df1_1.select(countDistinct(col(c))).collect()[0][0]
    if nunique <= max_categories:
        useful_categorical_cols.append(c)

print("\n카테고리 개수 20개 이하 유용 컬럼:", useful_categorical_cols)


### 범주형 변수별 결측치 비율 및 카테고리 개수

| 컬럼명                | 결측치비율 | 카테고리개수 |
|----------------------|------------|-------------|
| 기준년월              | 0.00%      | 6           |
| 발급회원번호          | 0.00%      | 2,976,855   |
| 남녀구분코드          | 0.00%      | 2           |
| 연령                  | 0.00%      | 6           |
| VIP등급코드           | 0.00%      | 5           |
| 최상위카드등급코드    | 0.00%      | 5           |
| 회원여부_이용가능     | 0.00%      | 2           |
| 회원여부_이용가능_CA  | 0.00%      | 2           |
| 회원여부_이용가능_카드론 | 0.00%   | 2           |
| 소지여부_신용         | 0.00%      | 2           |
| 회원여부_연체         | 0.00%      | 2           |
| 이용거절여부_카드론   | 0.00%      | 2           |
| 동의여부_한도증액안내 | 0.00%      | 2           |
| 수신거부여부_TM       | 0.00%      | 2           |
| 수신거부여부_DM       | 0.00%      | 2           |
| 수신거부여부_메일     | 0.00%      | 2           |
| 수신거부여부_SMS      | 0.00%      | 2           |
| 가입통신회사코드      | 0.00%      | 4           |
| 거주시도명            | 0.00%      | 17          |
| 직장시도명            | 0.00%      | 18          |
| 마케팅동의여부        | 0.00%      | 2           |
| Life_Stage           | 0.00%      | 7           |


**카테고리 개수 20개 이하 유용 컬럼:**  
`['기준년월', '남녀구분코드', '연령', 'VIP등급코드', '최상위카드등급코드', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '거주시도명', '직장시도명', '마케팅동의여부', 'Life_Stage']`

#축소후 완료된 데이터 정리 df1_2

In [0]:
selected_columns = [
    "발급회원번호",  # 회원 식별 컬럼 추가
    # 인구통계/고객 특성
    "남녀구분코드", "연령", "거주시도명", "직장시도명", "Life_Stage",
    "VIP등급코드", "최상위카드등급코드", "가입통신회사코드", "마케팅동의여부",
    # 카드 이용 이력/행동
    "입회경과개월수_신용", "최종카드발급경과월", "탈회횟수_누적", "카드신청건수",
    "소지카드수_유효_신용", "유효카드수_신용", "유효카드수_체크",
    "이용카드수_신용", "이용카드수_체크",
    "_1순위카드이용금액", "_1순위카드이용건수",
    "_2순위카드이용금액", "_2순위카드이용건수",
    "청구금액_기본연회비_B0M", "청구금액_제휴연회비_B0M",
    "기본연회비발생여부", "제휴연회비발생여부",
    # 리스크/충성도/이탈
    "회원여부_연체", "최종탈회후경과월", "탈회횟수_발급6개월이내", "탈회횟수_발급1년이내",
    "회원여부_이용가능", "동의여부_한도증액안내",
    "수신거부여부_TM", "수신거부여부_DM", "수신거부여부_메일", "수신거부여부_SMS"
]

# df1_1에서 필요한 컬럼만 추출하여 df1_2 생성
df1_2 = df1_1.select(selected_columns)

print(f"df1_2 컬럼 수: {len(df1_2.columns)}")
df1_2.printSchema()
df1_2.show(5, truncate=False)


In [0]:
from pyspark.sql.types import DoubleType
df1_2 = df1_2.withColumn("소지카드수_유효_신용", col("소지카드수_유효_신용").cast(DoubleType()))
# (다른 컬럼도 동일하게 반복)


In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

for c in ["유효카드수_신용", "유효카드수_체크", "이용카드수_신용", "이용카드수_체크"]:
    df1_2 = df1_2.withColumn(c, col(c).cast(DoubleType()))


In [0]:
# 1. 기존 테이블 및 경로 삭제
spark.sql("DROP TABLE IF EXISTS database_pjt.card_member_info_selected")
dbutils.fs.rm("dbfs:/user/hive/warehouse/database_pjt.db/card_member_info_selected", True)

# 2. 새로 저장
df1_2.write \
    .format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/user/hive/warehouse/database_pjt.db/card_member_info_selected") \
    .saveAsTable("database_pjt.card_member_info_selected")


In [0]:
print("저장된 테이블의 첫 5개 행:")
spark.sql("SELECT * FROM database_pjt.card_member_info_selected LIMIT 5").show()

print("저장된 테이블의 스키마:")
spark.sql("DESCRIBE database_pjt.card_member_info_selected").show()


## card_member_info_selected 가져와서 분석


In [0]:
# Delta Lake 테이블을 DataFrame으로 불러오기
df = spark.table("database_pjt.card_member_info_selected")
print(f"불러온 데이터 행 수: {df.count()}")
df.printSchema()
df.show(5, truncate=False)


In [0]:
df.describe().show()
df.groupBy("연령").count().orderBy("연령").show()
df.groupBy("Life_Stage").agg({"_1순위카드이용금액":"mean"}).show()


In [0]:
# 결측치 개수 DataFrame으로 만들어 Databricks display()로 표 형태 출력
import pandas as pd
missing_df = pd.DataFrame(list(missing_dict.items()), columns=['컬럼명', '결측치 개수'])
display(spark.createDataFrame(missing_df))


In [0]:
#항목별 개수,평균,최대,최소값
desc_pd = df.describe().toPandas().set_index('summary').T
print(desc_pd)


##df 설명값


%md
| 변수명 | count | mean | stddev | min | max |
|---|---|---|---|---|---|
| 남녀구분코드 | 17674485 | 1.4751 |  | 1.0 | 2.0 |
| 연령 | 17674485 |  |  | 20대 | 70대이상 |
| 거주시도명 | 17674485 |  |  | 강원 | 충북 |
| 직장시도명 | 17674485 |  |  | 강원 | 충북 |
| Life_Stage | 17674485 |  |  | 1.Single | 7.노령 |
| VIP등급코드 | 17674485 | 6.7254 |  | 04 | _ |
| 최상위카드등급코드 | 17674485 | 1.7711 |  | 1 | _ |
| 가입통신회사코드 | 17674485 |  |  | KTF | 알뜰폰 |
| 마케팅동의여부 | 17674485 | 0.8094 |  | 0.0 | 1.0 |
| 입회경과개월수_신용 | 17674485 | 73.9234 |  | 2.0 | 337.0 |
| 최종카드발급경과월 | 17674485 | 21.8454 |  | 0.0 | 59.0 |
| 탈회횟수_누적 | 17674485 | 0.5049 |  | 0.0 | 2.0 |
| 카드신청건수 | 17674485 | 0.0885 |  | 0.0 | 1.0 |
| 소지카드수_유효_신용 | 17674485 | 1.2672 |  | 0.0 | 4.0 |
| 유효카드수_신용 | 17674485 | 1.5514 |  | 0.0 | 5.0 |
| 유효카드수_체크 | 17674485 | 0.5675 |  | 0.0 | 3.0 |
| 이용카드수_신용 | 17674485 | 1.1928 |  | 0.0 | 5.0 |
| 이용카드수_체크 | 17674485 | 0.1612 |  | 0.0 | 2.0 |
| _1순위카드이용금액 | 17674485 | 1121190.5189 |  | -464393.0 | 12272789.0 |
| _1순위카드이용건수 | 17674485 | 40.0808 |  | -2.0 | 226.0 |
| _2순위카드이용금액 | 17674485 | 342266.9331 |  | -428324.0 | 8879045.0 |
| _2순위카드이용건수 | 17674485 | 14.5542 |  | -1.0 | 211.0 |
| 청구금액_기본연회비_B0M | 17674485 | 12.6088 |  | 0.0 | 11268.0 |
| 청구금액_제휴연회비_B0M | 17674485 | 35.2850 |  | 0.0 | 111896.0 |
| 기본연회비발생여부 | 17674485 | 0.0013 |  | 0.0 | 1.0 |
| 제휴연회비발생여부 | 17674485 | 0.0004 |  | 0.0 | 1.0 |
| 회원여부_연체 | 17674485 | 0.0175 |  | 0.0 | 1.0 |
| 최종탈회후경과월 | 17674485 | 27.5649 |  | 0.0 | 99.0 |
| 탈회횟수_발급6개월이내 | 17674485 | 0.0399 |  | 0.0 | 1.0 |
| 탈회횟수_발급1년이내 | 17674485 | 0.0628 |  | 0.0 | 1.0 |
| 회원여부_이용가능 | 17674485 | 0.9645 |  | 0.0 | 1.0 |
| 동의여부_한도증액안내 | 17674485 | 0.0999 |  | 0.0 | 1.0 |
| 수신거부여부_TM | 17674485 | 0.3564 |  | 0.0 | 1.0 |
| 수신거부여부_DM | 17674485 | 0.3146 |  | 0.0 | 1.0 |
| 수신거부여부_메일 | 17674485 | 0.3204 |  | 0.0 | 1.0 |
| 수신거부여부_SMS | 17674485 | 0.3705 |  | 0.0 | 1.0 |


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# 1. 수치형 변수 리스트 추출
numeric_cols = [c for c, t in df.dtypes if t in ['double', 'int']]

# 2. VectorAssembler로 features 컬럼 생성
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
df_vec = assembler.transform(df).select("features")

# 3. 피어슨 상관계수 행렬 계산
corr_matrix = Correlation.corr(df_vec, "features", "pearson").head()[0].toArray()

# 4. pandas DataFrame으로 변환해서 보기 좋게 출력
import pandas as pd
corr_df = pd.DataFrame(corr_matrix, index=numeric_cols, columns=numeric_cols)
print("수치형 변수 간 상관계수 행렬:")
print(corr_df.round(3))

# Databricks에서는 display()로 표 형태로도 확인 가능
display(corr_df)


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,10))
sns.heatmap(corr_df, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("수치형 변수 간 상관관계 히트맵")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


### 수치형 변수 간 상관관계 히트맵 해석

#### 1. 강한 양의 상관관계 (0.8 이상)
- **_2순위카드이용금액 ↔ _2순위카드이용건수** : 0.95  
  → 두 변수는 거의 완전히 같이 움직임. 정보 중복이 크므로 둘 중 하나만 남겨도 무방.
- **_1순위카드이용금액 ↔ _1순위카드이용건수** : 0.75  
  → 카드이용금액이 많을수록 이용건수도 많음(카드 사용량과 사용빈도는 밀접하게 연관).
- **_1순위카드이용금액 ↔ _2순위카드이용금액** : 0.73  
  → 1순위와 2순위 카드의 이용금액 간에도 강한 상관관계.
- **_1순위카드이용건수 ↔ _2순위카드이용건수** : 0.78  
  → 1순위와 2순위 카드의 이용건수도 강한 상관관계.

#### 2. 중간 정도의 양의 상관관계 (0.4~0.7)
- **유효카드수_신용 ↔ 소지카드수_유효_신용** : 0.67  
  → 신용카드 유효 보유수와 유효카드수는 비슷하게 움직임.
- **이용카드수_신용 ↔ 유효카드수_신용** : 0.69  
  → 실제 이용 신용카드 수와 유효 신용카드 수도 밀접하게 연관.
- **_1순위카드이용금액 ↔ 이용카드수_신용** : 0.49  
  → 카드 이용금액이 많을수록 실제 이용 카드 수도 많아지는 경향.

#### 3. 낮은 상관관계 (0.2 이하, 거의 없음)
- **입회경과개월수_신용, 최종카드발급경과월, 탈회횟수_누적, 카드신청건수 등**  
  → 대부분의 변수와 상관관계가 낮음. 독립적인 정보 제공.

#### 4. 연회비 관련 변수
- **청구금액_기본연회비_B0M ↔ 기본연회비발생여부** : 0.98  
  → 연회비 금액과 연회비 발생여부는 거의 완벽하게 일치(파생관계).
- **청구금액_제휴연회비_B0M ↔ 제휴연회비발생여부** : 1.00  
  → 제휴 연회비 금액과 발생여부도 완전히 일치.

#### 5. 특이점
- **_1순위, _2순위 카드 관련 변수들은 서로 중간~강한 상관관계**  
  → 카드 이용 패턴이 전반적으로 비슷하게 움직임.
- **연회비 관련 변수들은 대부분 다른 변수와는 상관관계가 거의 없음**  
  → 독립적인 특성.

#### 6. 정리 및 실무적 시사점
- **정보가 중복되는 변수(상관계수 0.8~1.0)는 컬럼 축소 시 하나만 남겨도 무방**
- **상관관계가 낮은 변수는 독립적 특성이므로 군집분석 등에서 정보 다양성 확보에 유리**
- **연회비 관련 파생 변수는 한 쪽만 남겨도 해석에 문제 없음**

---


다중공선성 확인용

In [0]:
# 수치형 컬럼만 추출
#numeric_cols = [c for c, t in df.dtypes if t in ['double', 'int']]
#df_pd = df.select(numeric_cols).toPandas()
#메모리 많다고 꺼져버림 ㅠ

In [0]:
# 1만건만 샘플링
numeric_cols = [c for c, t in df.dtypes if t in ['double', 'int']]

df_sample = df.select(numeric_cols).sample(False, 10000/df.count(), seed=42)
df_pd = df_sample.limit(10000).toPandas()


In [0]:
# VIF 계산


from statsmodels.stats.outliers_influence import variance_inflation_factor


vif_data = []
for i in range(len(df_pd.columns)):
    vif = variance_inflation_factor(df_pd.values, i)
    vif_data.append({'변수명': df_pd.columns[i], 'VIF': vif})

import pandas as pd
vif_df = pd.DataFrame(vif_data)
print(vif_df.sort_values('VIF', ascending=False))


In [0]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import matplotlib.pyplot as plt

# 2. VIF 계산
vif_data = []
for i in range(len(df_pd.columns)):
    vif = variance_inflation_factor(df_pd.values, i)
    vif_data.append({'변수명': df_pd.columns[i], 'VIF': vif})

vif_df = pd.DataFrame(vif_data).sort_values('VIF', ascending=False)

# 3. Spark DataFrame으로 변환(필요시)
vif_spark_df = spark.createDataFrame(vif_df)

# 4. VIF 시각화 (matplotlib 활용)
plt.figure(figsize=(10,6))
plt.barh(vif_df['변수명'], vif_df['VIF'], color='skyblue')
plt.xlabel('VIF')
plt.title('Variance Inflation Factor (VIF) by Variable')
plt.gca().invert_yaxis()  # 높은 VIF가 위로 오도록
plt.show()


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF 계산
vif_data = []
for i in range(len(df_pd.columns)):
    vif = variance_inflation_factor(df_pd.values, i)
    vif_data.append({'변수명': df_pd.columns[i], 'VIF': vif})

vif_df = pd.DataFrame(vif_data)

# 로그 변환 (log1p는 log(1 + x), 값이 0일 때도 안전함)
vif_df['log_VIF'] = np.log1p(vif_df['VIF'])

# 정렬
vif_df = vif_df.sort_values('log_VIF', ascending=False)

# 시각화
plt.figure(figsize=(10,6))
plt.barh(vif_df['변수명'], vif_df['log_VIF'], color='skyblue')
plt.xlabel('log(1 + VIF)')
plt.title('Log-Scaled Variance Inflation Factor (VIF)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


### 다중공선성(VIF) 분석 결과

| 변수명                | VIF           | 해석/비고                       |
|----------------------|---------------|----------------------------------|
| 청구금액_제휴연회비_B0M | 212,548,100   | **심각한 다중공선성 ** |
| 제휴연회비발생여부       | 212,534,100   | **심각한 다중공선성 ** |
| 유효카드수_신용         | 26.21         | 다중공선성 매우 높음      |
| 소지카드수_유효_신용     | 23.62         | 다중공선성 매우 높음       |
| _2순위카드이용건수      | 17.01         | 다중공선성 매우 높음      |
| _2순위카드이용금액      | 15.29         | 다중공선성 매우 높음      |
| 기본연회비발생여부       | 13.08         | 다중공선성 매우 높음     |
| 이용카드수_신용         | 9.85          | 다중공선성 높음            |
| 청구금액_기본연회비_B0M  | 9.83          | 다중공선성 높음              |
| _1순위카드이용건수      | 4.87          | 양호                                 |
| _1순위카드이용금액      | 4.72          | 양호                                 |
| 최종카드발급경과월       | 2.92          | 양호                                 |
| 입회경과개월수_신용      | 2.45          | 양호                                 |
| 유효카드수_체크         | 2.17          | 양호                                 |
| 이용카드수_체크         | 1.72          | 양호                                 |
| 탈회횟수_누적           | 1.51          | 양호                                 |
| 카드신청건수            | 1.17          | 양호                                 |



다중공선성이 높은 항목은 존재하지만, 파생변수인점, 

서로 누가봐도 연관관계가 있는 변수끼리 영향을 끼쳐서

그렇다는 점을 제외하면 문제될게 없다.

로그변환전 분포 왜도 확인(분포 치우친 정도 )

In [0]:
import matplotlib.pyplot as plt
import math

num_plots_per_page = 4
num_cols_len = len(numeric_cols)
num_pages = math.ceil(num_cols_len / num_plots_per_page)

for page in range(num_pages):
    start_idx = page * num_plots_per_page
    end_idx = min(start_idx + num_plots_per_page, num_cols_len)
    cols_subset = numeric_cols[start_idx:end_idx]
    
    fig, axes = plt.subplots(nrows=1, ncols=num_plots_per_page, figsize=(6*num_plots_per_page, 4))
    
    # axes가 1개일 때도 리스트로 변환
    if num_plots_per_page == 1:
        axes = [axes]
    
    # 실제 데이터가 있는 축만 히스토그램 그림
    for i, col in enumerate(cols_subset):
        df_pd[col].hist(bins=50, ax=axes[i])
        axes[i].set_title(col)
    
    # 남은 subplot(빈 축)은 숨김
    for j in range(len(cols_subset), num_plots_per_page):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()


### 주요 수치형 변수 분포 해석 (1~17번, 히스토그램 기반)
(필요한번호는 1 10 11 12 13)
#### 1. 입회경과개월수_신용
- 오른쪽(positive)으로 긴 꼬리가 있는 전형적인 치우친 분포
- **로그 변환 권장**  
- 대부분 고객이 가입 후 경과 개월이 짧고, 일부만 매우 길다

#### 2. 최종카드발급경과월
- 비교적 완만한 분포, 특정 구간에 빈도 집중
- **로그 변환 필요성 낮음**

#### 3. 탈회횟수_누적
- 대부분 0, 일부만 1~2 (이산형, 한쪽에 몰림)
- **로그 변환 불필요**  
- 그대로 이진/이산 변수로 사용

#### 4. 카드신청건수
- 거의 대부분 0, 일부만 1 (이진형)
- **로그 변환 불필요**

#### 5. 소지카드수_유효_신용
- 1에 빈도 집중, 일부만 2~3 (이산형)
- **로그 변환 불필요**

#### 6. 유효카드수_신용
- 1~2에 빈도 집중, 3~5는 소수
- **로그 변환 불필요**

#### 7. 유효카드수_체크
- 0~1에 빈도 집중, 2~3은 소수
- **로그 변환 불필요**

#### 8. 이용카드수_신용
- 1에 빈도 집중, 2~4는 소수
- **로그 변환 불필요**

#### 9. 이용카드수_체크
- 대부분 0, 일부만 1 (이진형)
- **로그 변환 불필요**

#### 10. _1순위카드이용금액
- 극단적으로 오른쪽으로 치우친 분포(극단값 많음)
- **로그 변환 강력 권장**  
- 대부분 소액, 일부만 고액

#### 11. _1순위카드이용건수
- 오른쪽 꼬리, 치우친 분포
- **로그 변환 권장**

#### 12. _2순위카드이용금액
- 대부분 0~소액, 극소수만 고액
- **로그 변환 권장**

#### 13. _2순위카드이용건수
- 오른쪽 꼬리, 치우친 분포
- **로그 변환 권장**

#### 14. 청구금액_기본연회비_B0M
- 거의 모든 값이 0, 일부만 큰 값(연회비 발생 고객 소수)
- **로그 변환 효과 없음**  
- 이진 변수로 처리하거나 그대로 사용

#### 15. 청구금액_제휴연회비_B0M
- 거의 모든 값이 0, 일부만 큰 값(제휴 연회비 발생 고객 소수)
- **로그 변환 효과 없음**

#### 16. 기본연회비발생여부
- 대부분 0, 극소수만 1 (이진형)
- **로그 변환 불필요**

#### 17. (추가 변수)  
- 이미지에 포함된 17번째 변수도 위와 동일한 기준으로 해석 가능  
- 대부분 이진/이산형이거나, 극단적으로 치우친 분포는 로그 변환 효과 없음

---

#### **정리**
- **로그 변환 권장:**  
  - 입회경과개월수_신용, _1순위카드이용금액, _1순위카드이용건수, _2순위카드이용금액, _2순위카드이용건수
- **로그 변환 불필요:**  
  - 이진/이산 변수(탈회횟수_누적, 카드신청건수, 소지카드수_유효_신용 등), 연회비 관련 변수

- 이진/이산 변수는 그대로 사용  
- 연속형, 극단값 많은 변수는 로그 변환 후 군집분석/모델링에 활용

---


### 로그 변환이 필요한 변수 요약 및 해석

#### 1. 입회경과개월수_신용
- 오른쪽(positive)으로 꼬리가 긴 치우친 분포
- 대부분 고객은 가입 후 경과 개월이 짧고, 일부만 매우 길다
- **로그 변환 적용 시 분포가 더 대칭에 가까워져 군집분석, 모델링에 유리**

#### 2. _1순위카드이용금액
- 극단적으로 오른쪽으로 치우친 분포(극단값 다수)
- 대부분 소액, 일부만 고액 이용
- **로그 변환 적용 시 이상치 영향이 줄고, 분포가 안정됨**

#### 3. _1순위카드이용건수
- 오른쪽 꼬리, 치우친 분포
- 카드 사용 빈도가 소수에 몰려 있고, 일부만 매우 많음
- **로그 변환 적용 시 분포가 더 균등해짐**

#### 4. _2순위카드이용금액
- 대부분 0~소액, 극소수만 고액
- **로그 변환 적용 시 극단값 영향 감소, 분포 개선**

#### 5. _2순위카드이용건수
- 오른쪽 꼬리, 치우친 분포
- **로그 변환 적용 시 분포가 더 대칭적으로 변함**

---

#### **정리**
- 위 5개 변수는 모두 연속형 수치 데이터이며,  
  **값이 0~소수에 몰려 있고, 극단적으로 큰 값이 일부 존재하는 전형적인 치우친 분포**
- **로그 변환(log1p) 적용을 강력히 권장**,  
  변환 후 군집분석/모델링의 성능과 해석력이 향상가능

---
설명력을 잃지 않을까 고민햇지만 
로그변환을 하면 소수의 큰값이 압축되어 더 고른 변화
참고로 아래껀 로그변환한걸 새로 추가한 개념


In [0]:
#로그변환
from pyspark.sql.functions import log1p

log_cols = [
    "입회경과개월수_신용",
    "_1순위카드이용금액",
    "_1순위카드이용건수",
    "_2순위카드이용금액",
    "_2순위카드이용건수"
]

for col_name in log_cols:
    df = df.withColumn(f"log_{col_name}", log1p(col_name))



In [0]:
# 로그변환 컬럼
log_cols = [
    "log_입회경과개월수_신용",
    "log__1순위카드이용금액",
    "log__1순위카드이용건수",
    "log__2순위카드이용금액",
    "log__2순위카드이용건수"
]

# 기본 수치형 컬럼 (이진/이산형, 로그변환 불필요)
base_cols = [
    "최종카드발급경과월",
    "탈회횟수_누적",
    "카드신청건수",
    "소지카드수_유효_신용",
    "유효카드수_신용",
    "유효카드수_체크",
    "이용카드수_신용",
    "이용카드수_체크",
    "기본연회비발생여부",
    "제휴연회비발생여부"
]


In [0]:
# 2. 로그변환 컬럼과 기본 컬럼의 null 값 확인 및 처리
# (필요시 null이 있는 행 삭제)
df = df.na.drop(subset=log_cols + base_cols)

# 3. 로그변환 변수 벡터화 및 표준화
assembler_log = VectorAssembler(
    inputCols=log_cols,
    outputCol="log_features",
    handleInvalid="keep"
)
df_log = assembler_log.transform(df)

scaler_log = StandardScaler(
    inputCol="log_features",
    outputCol="scaled_log_features",
    withMean=True,
    withStd=True
)
df_log_scaled = scaler_log.fit(df_log).transform(df_log)


In [0]:
# 4. 기본 변수 벡터화 및 표준화
assembler_base = VectorAssembler(
    inputCols=base_cols,
    outputCol="base_features",
    handleInvalid="keep"
)
df_base = assembler_base.transform(df_log_scaled)

scaler_base = StandardScaler(
    inputCol="base_features",
    outputCol="scaled_base_features",
    withMean=True,
    withStd=True
)
df_all_scaled = scaler_base.fit(df_base).transform(df_base)

In [0]:
# 5. 최종 Feature Vector 합치기
final_assembler = VectorAssembler(
    inputCols=["scaled_log_features", "scaled_base_features"],
    outputCol="final_features"
)
df_final = final_assembler.transform(df_all_scaled)

# df_final의 "final_features" 컬럼을 군집분석 등에서 사용

In [0]:
spark.sql("DROP TABLE IF EXISTS database_pjt.df_final")
df_final.write.format("delta").mode("overwrite").saveAsTable("database_pjt.df_final")


In [0]:
# 저장 후 테이블 목록 확인
spark.sql(f"SHOW TABLES IN {db_name}").show()

# 저장된 테이블 불러오기 및 샘플 확인
df_final = spark.read.format("delta").table(table_name)
df_final.show(3)


# 군집분석

In [0]:
print(df_final.columns)


In [0]:
df_final.select("final_features").show(truncate=False)



다중공선성 체크해본 변수

청구금액_제휴연회비_B0M /제휴연회비발생여부

유효카드수_신용 / 소지카드수_유효_신용/이용카드수_신용

_2순위카드이용건수/_2순위카드이용금액

기본연회비발생여부/청구금액_기본연회비_B0M

In [0]:
# 돌리지 말것 돌리더라도 23분걸리니 얼른 취소할것
# 모든행 다 검사 결과 결측치도 무한대도 ㅇ벗어 그럼 그냥 단순 메모리 이슈?

from pyspark.sql.functions import col, sum

# 결측치 확인
null_counts = df_final.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_final.columns])
null_counts.show(truncate=False)

# 무한대 값 확인 (로그 변환에서 발생한 -inf/inf)
inf_check = df_final.select([sum(col(c).isin(float('inf'), -float('inf')).cast("int")).alias(c) for c in log_cols])
inf_check.show(truncate=False)


In [0]:
#돌리지마
#대참사다  클러스터링하는데 메모리 부족 이슈남 어카지? 샘플링 20퍼 10퍼 해서해야나?
# K=4로 한번
from pyspark.ml.clustering import KMeans

k = 4 
kmeans = KMeans(featuresCol="final_features", k=k, seed=42)
model = kmeans.fit(df_final)
df_clustered = model.transform(df_final)

# 군집별 인원수, 평균 등 프로파일링
df_clustered.groupBy("prediction").count().show()
df_clustered.groupBy("prediction").mean().show()


In [0]:
# 2. 데이터 캐싱
df_clustering = df_final.select("final_features").cache()
df_clustering.count()

# 3. K-means 파라미터 설정
kmeans = KMeans(
    featuresCol="final_features",
    k=4,
    initMode="k-means||",
    initSteps=5,
    maxIter=50,
    seed=42
)

# 4. 모델 학습
model = kmeans.fit(df_clustering)

# 5. 캐싱 해제
df_clustering.unpersist()

In [0]:
# 10% 샘플 추출
df_sample = df_final.select("final_features").sample(False, 0.1, seed=42).cache()
df_sample.count()  # 캐싱 트리거

# K-means 모델 학습
kmeans = KMeans(
    featuresCol="final_features",
    k=4,
    initMode="k-means||",
    initSteps=5,
    maxIter=50,
    seed=42
)
model = kmeans.fit(df_sample)

# 샘플 데이터 정리
df_sample.unpersist()


In [0]:
#최적값 찾기
from pyspark.ml.evaluation import ClusteringEvaluator

silhouette_scores = []
for k in range(2, 9):
    kmeans = KMeans(featuresCol="final_features", k=k, seed=42)
    model = kmeans.fit(df_final)
    predictions = model.transform(df_final)
    evaluator = ClusteringEvaluator(featuresCol="final_features", metricName="silhouette", distanceMeasure="squaredEuclidean")
    score = evaluator.evaluate(predictions)
    silhouette_scores.append((k, score))
    print(f"k={k}, silhouette score={score}")




# --------- 0602 김세빈 데이터 전처리 ( 상원씨 자료 이어받아 해보기 !) -----

### 데이터 전처리 방향

1.이용금액을 총 금액으로 선정 및 이용건수는 삭제
 - 1순위 이용금액 및 2순위 이용금액, 이용건수등의 상관관계를 보고 둘의 상관관계성이 높아 삭제해야된다고 판단
 - 건수보다는구체적인 금액을 제시하는 쪽이 소비 데이터를 나타낸다고 판단
 - 이때 1순위, 2순위 이용금액에 대한 내용은 카드 수에 따라 금액을 달리 보여줄 필요가 있으므로 일단 "총 이용금액"으로서 1순위 2순위 이용금액을 집계할 것
\t

2.기본연회비 발생여부, 제휴연회비 발생여부 삭제
 - 파생컬럼 삭제
\t

3.유효카드수_신용 삭제 
 - 상관관계 및 다중공선성을 고려할때 삭제하고 가기로 판단, 대체 항목이 있다고 생각
\t

4.입회 경과 개월수_신용,1순위카드이용금액(총이용금액)은 로그 변환 할 것, 이때 로그는 자연로그로 지정 

5.샘플링 없이 인코딩 후 PCA 과정을 수행한 뒤 클러스터링 진행

In [0]:
# Delta Lake 테이블을 DataFrame으로 불러오기
df = spark.table("database_pjt.card_member_info_selected")
print(f"불러온 데이터 행 수: {df.count()}")
df.printSchema()
df.show(5, truncate=False)


In [0]:
df.columns

In [0]:
# 이용금액을 총 금액으로 선정 및 이용건수,기본연회비 발생여부, 제휴연회비 발생여부,유효카드수_신용 삭제

delete_col = ['_1순위카드이용건수','_2순위카드이용건수','기본연회비발생여부','제휴연회비발생여부','유효카드수_신용']
df = df.drop(*delete_col)

df.columns

In [0]:
# 1순위 및 2순위 이용금액을 총 이용금액이라는 새로운 컬럼으로 파생
from pyspark.sql.functions import col

# 1순위 및 2순위 이용금액을 총 이용금액이라는 새로운 컬럼으로 파생
df = df.withColumn('총이용금액', col('_1순위카드이용금액') + col('_2순위카드이용금액'))
df = df.drop('_1순위카드이용금액', '_2순위카드이용금액')

display(df.limit(3))
    

In [0]:
## 각 칼럼들의 데이터 형태 확인

df.describe()

In [0]:
# 수치형 없나 다시 확인

numeric_cols = [c for c, t in df.dtypes if t in ['double', 'int']]

numeric_cols

In [0]:
#로그변환
from pyspark.sql.functions import log1p

log_cols = [
    "입회경과개월수_신용",
    "총이용금액",
]

for col_name in log_cols:
    df = df.withColumn(f"log_{col_name}", log1p(col_name))

df.drop('입회경과개월수_신용','총이용금액')

### 추가 전처리 작업

1. 범주형 변수 인코딩 (if 존재)   
-Label Encoding: 순서가 있는 범주 (예: 등급, 점수)   
-One-Hot Encoding: 순서 없는 범주 (예: 성별, 지역)

2.차원 축소 (PCA, t-SNE 등)

In [0]:
display(df.limit(2))

In [0]:
df.drop("총이용금액")

In [0]:
from pyspark.sql.functions import when, col

df = df.withColumn(
    "최상위카드등급코드",
    when(col("최상위카드등급코드") == "-", "5").otherwise(col("최상위카드등급코드"))
)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# 1. 인코딩 대상 컬럼 정의
one_hot_cols = [
    "남녀구분코드", "거주시도명", "직장시도명", "Life_Stage",
    "마케팅동의여부", "회원여부_연체", "회원여부_이용가능",
    "동의여부_한도증액안내", "수신거부여부_TM", "수신거부여부_메일", "수신거부여부_SMS","가입통신회사코드",
]
label_cols = ["VIP등급코드", "연령","최상위카드등급코드"]

# 2. Indexing + One-Hot Encoding 단계 정의
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep")
    for col in one_hot_cols + label_cols
]

onehots = [
    OneHotEncoder(inputCol=f"{col}_idx", outputCol=f"{col}_oh", handleInvalid="keep")
    for col in one_hot_cols
]

# 3. 파이프라인 실행
pipeline = Pipeline(stages=indexers + onehots)
encoded_model = pipeline.fit(df)
df_encoded = encoded_model.transform(df)

# 4. 사용할 컬럼 정리
encoded_cols = [f"{col}_oh" for col in one_hot_cols if f"{col}_oh" in df_encoded.columns] + \
               [f"{col}_idx" for col in label_cols if f"{col}_idx" in df_encoded.columns]

original_cols = [col for col in df.columns if col not in (one_hot_cols + label_cols)]

# 5. 최종 df 구성
df_final = df_encoded.select(original_cols + encoded_cols)


In [0]:
from pyspark.sql.functions import col

df_final = df_final.withColumn("최종탈회후경과월", col("최종탈회후경과월").cast("int")) \
       .withColumn("탈회횟수_발급6개월이내", col("탈회횟수_발급6개월이내").cast("int")) \
       .withColumn("탈회횟수_발급1년이내", col("탈회횟수_발급1년이내").cast("int"))


In [0]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# 기본 수치형 컬럼들 정의 (로그변환 등 하지 않은 일반 수치형 변수들)
base_cols = ["입회경과개월수_신용","최종카드발급경과월","탈회횟수_누적","카드신청건수","소지카드수_유효_신용","유효카드수_체크","이용카드수_신용","이용카드수_체크","청구금액_기본연회비_B0M","청구금액_제휴연회비_B0M","최종탈회후경과월","탈회횟수_발급6개월이내","탈회횟수_발급1년이내","log_입회경과개월수_신용","log_총이용금액"]  # <- 여기에 실제 사용 변수명 삽입

df_final = df_final.na.drop(subset=base_cols)

# 1. 벡터화
assembler_base = VectorAssembler(
    inputCols=base_cols,
    outputCol="base_features"
)
df_base_vectorized = assembler_base.transform(df_final)  # df_final: 인코딩된 최종 데이터프레임

# 2. 표준화
scaler_base = StandardScaler(
    inputCol="base_features",
    outputCol="scaled_base_features",
    withMean=True,
    withStd=True
)
df_base_scaled = scaler_base.fit(df_base_vectorized).transform(df_base_vectorized)


In [0]:
# 인코딩 후 생성된 one-hot, label index 컬럼을 결합할 대상 지정
existing_encoded_cols = [col for col in encoded_feature_cols if col in df_base_scaled.columns]

assembler_encoded = VectorAssembler(
    inputCols=existing_encoded_cols,
    outputCol="encoded_features"
)
df_encoded_vectorized = assembler_encoded.transform(df_base_scaled)



In [0]:
# 최종적으로 사용할 모든 features = scaled 수치형 + 인코딩 벡터
final_feature_cols = ["scaled_base_features", "encoded_features"]

assembler_all = VectorAssembler(
    inputCols=final_feature_cols,
    outputCol="final_features"
)
df_final = assembler_all.transform(df_encoded_vectorized)

## PCA 처리

In [0]:
from pyspark.ml.feature import VectorAssembler

# (1) 실제 존재하는 one-hot 컬럼만 필터링
encoded_feature_cols = [f"{col}_oh" for col in one_hot_cols]
existing_encoded_cols = [col for col in encoded_feature_cols if col in df_final.columns]

# (2) VectorAssembler 구성
assembler = VectorAssembler(
    inputCols=existing_encoded_cols,
    outputCol="features"
)

df_vectorized = assembler.transform(df_final)


In [0]:
#2단계 : PCA 적용

from pyspark.ml.feature import PCA

# PCA 적용 (K 초깃값 5)
pca = PCA(k=25, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_vectorized)
df_pca = pca_model.transform(df_vectorized)


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspark.ml.feature import PCA

# 1. PCA 모델 학습
pca = PCA(k=25, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_vectorized)

# 2. 설명 분산 비율 추출
explained_variance = pca_model.explainedVariance.toArray()

# 3. 누적 설명 분산 계산
cumulative_variance = np.cumsum(explained_variance)

# 4. 시각화를 위한 Pandas DataFrame 생성
pca_df = pd.DataFrame({
    'PC': [f'PC{i+1}' for i in range(len(explained_variance))],
    'Explained Variance': explained_variance,
    'Cumulative Variance': cumulative_variance
})

# 5. 시각화
plt.figure(figsize=(10,6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-')
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.xticks(ticks=range(1, len(cumulative_variance) + 1))
plt.ylim(0, 1.05)
plt.tight_layout()
plt.show()


In [0]:
#3단계 : 결과 확인

# 결과 확인: 주성분 벡터 출력
df_pca.select("pca_features").show(5, truncate=False)


In [0]:
# 적절한 K갯수 확인 

# 설명 분산 추출
explained_variance = pca_model.explainedVariance.toArray()
cumulative_variance = explained_variance.cumsum()

# 몇 개의 주성분으로 몇 %까지 설명하는지 출력
for i, cum_var in enumerate(cumulative_variance):
    print(f"Top {i+1} PCs: {cum_var:.4f}")

### K 5일때, 누적분산 비율이 54%까지 밖에 설명하지 못하므로 K갯수를 늘려야함. (합격 기준은 95%로 설정)

In [0]:

# PCA 적용 (K를 10으로 설정)
pca = PCA(k=10, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_vectorized)
df_pca = pca_model.transform(df_vectorized)


In [0]:
# 적절한 K갯수 확인 

# 설명 분산 추출
explained_variance = pca_model.explainedVariance.toArray()
cumulative_variance = explained_variance.cumsum()

# 몇 개의 주성분으로 몇 %까지 설명하는지 출력
for i, cum_var in enumerate(cumulative_variance):
    print(f"Top {i+1} PCs: {cum_var:.4f}")

In [0]:
# PCA 적용 (K를 15으로 설정)
pca = PCA(k=15, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_vectorized)
df_pca = pca_model.transform(df_vectorized)


In [0]:
# 적절한 K갯수 확인 

# 설명 분산 추출
explained_variance = pca_model.explainedVariance.toArray()
cumulative_variance = explained_variance.cumsum()

# 몇 개의 주성분으로 몇 %까지 설명하는지 출력
for i, cum_var in enumerate(cumulative_variance):
    print(f"Top {i+1} PCs: {cum_var:.4f}")

In [0]:
# PCA 적용 (K를 25으로 설정)
pca = PCA(k=25, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_vectorized)
df_pca = pca_model.transform(df_vectorized)


In [0]:
# 적절한 K갯수 확인 

# 설명 분산 추출
explained_variance = pca_model.explainedVariance.toArray()
cumulative_variance = explained_variance.cumsum()

# 몇 개의 주성분으로 몇 %까지 설명하는지 출력
for i, cum_var in enumerate(cumulative_variance):
    print(f"Top {i+1} PCs: {cum_var:.4f}")

In [0]:
df_pca.columns

#데이터 중간저장


In [0]:
## final 기준 인코딩된 칼럼 확인

df_final.columns

In [0]:
## ML용 컬럼 떼어버리기

df_final = df_final.drop('base_features','scaled_base_features','encoded_features','final_features')

df_final.columns

컬럼 분류

+전부회원번호추가요망

🔹 1. 회원 기본 정보 테이블 (Member)   
컬럼명   
남녀구분코드_oh   
연령_idx   
거주시도명_oh   
직장시도명_oh   
Life_Stage_oh   
회원여부_연체_oh   
회원여부_이용가능_oh   
VIP등급코드_idx   
최상위카드등급코드_idx   

🔹 2. 카드 보유 및 이용 이력 테이블 (Card_Usage)   
컬럼명   
소지카드수_유효_신용   
유효카드수_체크   
이용카드수_신용   
이용카드수_체크   
총이용금액   
log_총이용금액   

🔹 3. 카드 신청 및 탈회 이력 테이블 (Card_History)   
컬럼명   
카드신청건수   
탈회횟수_누적   
탈회횟수_발급6개월이내   
탈회횟수_발급1년이내   
최종탈회후경과월   

🔹 4. 회원 가입 및 활동 테이블 (Membership)   
컬럼명   
입회경과개월수_신용   
log_입회경과개월수_신용   
최종카드발급경과월   

🔹 5. 청구 관련 테이블 (Billing)   
컬럼명   
청구금액_기본연회비_B0M   
청구금액_제휴연회비_B0M   

🔹 6. 마케팅/동의/수신 여부 테이블 (Marketing_Consent)   
컬럼명   
마케팅동의여부_oh   
동의여부_한도증액안내_oh   
수신거부여부_DM   
수신거부여부_TM_oh   
수신거부여부_메일_oh   
수신거부여부_SMS_oh   

🔹 7. 인덱싱/파생 컬럼 (보조 테이블 or 파생용)   
컬럼명   
log_입회경과개월수_신용   
log_총이용금액   

In [0]:
df_final.write.format("delta").mode("error").saveAsTable("database_pjt.df_final_final")
#이거 뭐였지?

In [0]:
spark.sql("DROP TABLE IF EXISTS database_pjt.1_fina_member")
spark.sql("DROP TABLE IF EXISTS database_pjt.1_fina_card_usage")
spark.sql("DROP TABLE IF EXISTS database_pjt.1_fina_card_history")

In [0]:
member_cols = [
    "발급회원번호","남녀구분코드_oh", "연령_idx", "거주시도명_oh", "직장시도명_oh", "Life_Stage_oh", 
    "회원여부_연체_oh", "회원여부_이용가능_oh", "VIP등급코드_idx", "최상위카드등급코드_idx"
]

card_usage_cols = [
    "발급회원번호","소지카드수_유효_신용", "유효카드수_체크", "이용카드수_신용", "이용카드수_체크", "총이용금액", "log_총이용금액"
]

card_history_cols = [
    "발급회원번호","카드신청건수", "탈회횟수_누적", "탈회횟수_발급6개월이내", "탈회횟수_발급1년이내", "최종탈회후경과월"
]


df_1_fina_member = df_final.select(member_cols)
df_1_fina_card_usage = df_final.select(card_usage_cols)
df_1_fina_card_history = df_final.select(card_history_cols)


df_1_fina_member.write.format("delta").mode("overwrite").saveAsTable("database_pjt.1_fina_member")
df_1_fina_card_usage.write.format("delta").mode("overwrite").saveAsTable("database_pjt.1_fina_card_usage")
df_1_fina_card_history.write.format("delta").mode("overwrite").saveAsTable("database_pjt.1_fina_card_history")


In [0]:
#회원정보 넣는라 삭제중
spark.sql("DROP TABLE IF EXISTS hive_metastore.database_pjt.1_fina_Membership")
spark.sql("DROP TABLE IF EXISTS hive_metastore.database_pjt.1_fina_Billing")
spark.sql("DROP TABLE IF EXISTS hive_metastore.database_pjt.1_Marketing_Consent")
spark.sql("DROP TABLE IF EXISTS hive_metastore.database_pjt.1_fina_log")



In [0]:
spark.sql("DROP TABLE IF EXISTS hive_metastore.database_pjt.1_Marketing_Consent")

In [0]:
#🔹 4. 회원 가입 및 활동 테이블 (Membership)
# 저장할 컬럼 선택
membership_columns = [
    '발급회원번호',
    '입회경과개월수_신용',
    'log_입회경과개월수_신용',
    '최종카드발급경과월'
]

# 해당 컬럼만 포함한 DataFrame 생성
membership_df = df_final.select(membership_columns)

# Hive metastore > database_pjt > 1_fina_Membership에 테이블로 저장
membership_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("hive_metastore.database_pjt.1_fina_Membership")

In [0]:
#🔹 5. 청구 관련 테이블 (Billing)
# 저장할 컬럼 선택
Billing_columns = [
    '발급회원번호',
    '청구금액_기본연회비_B0M',
    '청구금액_제휴연회비_B0M',
]

# 해당 컬럼만 포함한 DataFrame 생성
Billing_df = df_final.select(Billing_columns)

# Hive metastore > database_pjt > 1_fina_Billing에 테이블로 저장
Billing_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("hive_metastore.database_pjt.1_fina_Billing")

In [0]:
#🔹  6. 마케팅/동의/수신 여부 테이블 (Marketing_Consent)
# 저장할 컬럼 선택
Marketing_Consent = [
    '발급회원번호',
    '마케팅동의여부_oh',
    '동의여부_한도증액안내_oh',
    '수신거부여부_DM',
    '수신거부여부_TM_oh',
    '수신거부여부_메일_oh',
    '수신거부여부_SMS_oh'
]

# 해당 컬럼만 포함한 DataFrame 생성
Marketing_Consent = df_final.select(Marketing_Consent)

# Hive metastore > database_pjt > 1_fina_Billing에 테이블로 저장
Marketing_Consent.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("hive_metastore.database_pjt.1_fina_Marketing_Consent")

In [0]:
#🔹 7. 인덱싱/파생 컬럼 (보조 테이블 or 파생용, log)
# 저장할 컬럼 선택
log_columns = [
    '발급회원번호',
    'log_입회경과개월수_신용',
    'log_총이용금액',
]

# 해당 컬럼만 포함한 DataFrame 생성
log_df = df_final.select(log_columns)

# Hive metastore > database_pjt > 1_fina_log에 테이블로 저장
log_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("hive_metastore.database_pjt.1_fina_log")

In [0]:
%sql
SELECT * FROM hive_metastore.database_pjt.`1_Marketing_Consent` LIMIT 10;


In [0]:
# 1. 회원 기본 정보 테이블 불러오기
member_df = spark.table("hive_metastore.database_pjt.1_fina_member")
member_df.show(5, truncate=False)

# 2. 카드 보유 및 이용 이력 테이블 불러오기
card_usage_df = spark.table("hive_metastore.database_pjt.1_fina_card_usage")
card_usage_df.show(5, truncate=False)

# 3. 카드 신청 및 탈회 이력 테이블 불러오기
card_history_df = spark.table("hive_metastore.database_pjt.1_fina_card_history")
card_history_df.show(5, truncate=False)

# 4. 회원 가입 및 활동 테이블 불러오기
membership_df = spark.table("hive_metastore.database_pjt.1_fina_Membership")
membership_df.show(5, truncate=False)

# 5. 청구 관련 테이블 불러오기
billing_df = spark.table("hive_metastore.database_pjt.1_fina_Billing")
billing_df.show(5, truncate=False)

# 6. 마케팅/동의/수신 여부 테이블 불러오기
marketing_consent_df = spark.table("hive_metastore.database_pjt.1_Marketing_Consent")
marketing_consent_df.show(5, truncate=False)

# 7. 인덱싱/파생 컬럼 테이블 불러오기
log_df = spark.table("hive_metastore.database_pjt.1_fina_log")
log_df.show(5, truncate=False)


### 클러스터링 - 앙상블

1. 선정 모델
- KMeans
- GaussianMixture
- BisectingKMeans

In [0]:
pip install torch kmeans-pytorch


In [0]:
from pyspark.sql.functions import col, isnan

df_pca.select("pca_features").where(col("pca_features").isNull()).count()


In [0]:
from pyspark.ml.clustering import KMeans, GaussianMixture, BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# 10% 샘플링, withReplacement=False는 비복원 추출, seed는 재현성을 위해 설정
df_sampled = df_pca.sample(withReplacement=False, fraction=0.01, seed=42)

# 2. 각 모델 학습
kmeans_model = KMeans(k=10, featuresCol="pca_features").fit(df_pca)
gmm_model = GaussianMixture(k=10, featuresCol="pca_features").fit(df_pca)
bkm_model = BisectingKMeans(k=10, featuresCol="pca_features").fit(df_pca)

# 3. 클러스터링 결과 저장
df_kmeans = kmeans_model.transform(df_pca).withColumnRenamed("prediction", "kmeans_cluster")
df_gmm = gmm_model.transform(df_pca).withColumnRenamed("prediction", "gmm_cluster")
df_bkm = bkm_model.transform(df_pca).withColumnRenamed("prediction", "bkm_cluster")

# 4. 병합 및 메타 클러스터링 (ID를 기준으로 조인)
df_ensemble = df_kmeans.select("id", "kmeans_cluster") \
    .join(df_gmm.select("id", "gmm_cluster"), on="id") \
    .join(df_bkm.select("id", "bkm_cluster"), on="id")

# 5. 클러스터 조합을 features로 변환 (OneHotEncoder or VectorAssembler)
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["kmeans_cluster", "gmm_cluster", "bkm_cluster"], outputCol="meta_features")
df_meta = assembler.transform(df_ensemble)

# 6. 메타 클러스터링
meta_model = KMeans(k=5, featuresCol="meta_features", predictionCol="final_cluster").fit(df_meta)
df_final_cluster = meta_model.transform(df_meta)
 

## 파이토치

In [0]:
# pip 설치
!pip install torch kmeans-pytorch

# conda 설치
!conda install -c rapidsai -c nvidia -c conda-forge cuml=24.04 -y


In [0]:
import torch
import numpy as np
from kmeans_pytorch import kmeans

# 1. PySpark DataFrame → Numpy → PyTorch Tensor (GPU)
pca_features_np = np.array([row.pca_features.toArray() for row in df_pca.select("pca_features").collect()])
pca_features = torch.from_numpy(pca_features_np).float().cuda()

# 2. 10% 샘플링 (비복원 추출)
sample_size = int(0.1 * len(pca_features))
sample_indices = torch.randperm(len(pca_features))[:sample_size]
pca_sampled = pca_features[sample_indices]

# 3. KMeans 모델 학습 (GPU)
kmeans_cluster_ids, kmeans_centers = kmeans(
    X=pca_sampled,
    num_clusters=5,
    distance='euclidean',
    device=torch.device('cuda')
)

# 4. 전체 데이터에 KMeans 예측 (GPU)
kmeans_cluster_ids_full = kmeans.predict(pca_features)

# 5. Bisecting KMeans 대신 추가 KMeans 모델 (GPU)
kmeans2_cluster_ids, kmeans2_centers = kmeans(
    X=pca_sampled,
    num_clusters=5,
    distance='cosine',
    device=torch.device('cuda')
)
kmeans2_cluster_ids_full = kmeans2.predict(pca_features)

# 6. 메타 특징 생성 (GPU)
meta_features = torch.stack([kmeans_cluster_ids_full, kmeans2_cluster_ids_full], dim=1).float()

# 7. 메타 클러스터링 (GPU)
meta_cluster_ids, meta_centers = kmeans(
    X=meta_features,
    num_clusters=5,
    distance='euclidean',
    device=torch.device('cuda')
)  

# 8. 결과를 PySpark DataFrame으로 변환
from pyspark.sql import Row

final_cluster_ids_np = meta_cluster_ids.cpu().numpy().astype(int)
rows = [Row(id=idx, final_cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)

df_final_cluster.show(5)


In [0]:
#2-1. KMeans (GPU)
# KMeans 클러스터링 (k=10)
kmeans_cluster_ids, kmeans_centers = kmeans(
    X=pca_sampled,
    num_clusters=10,
    distance='euclidean',
    device=torch.device('cuda'),
    batch_size=512  # 대용량 데이터 시 배치 처리
)

# 전체 데이터 예측
kmeans_cluster_ids_full = kmeans.predict(pca_features)


In [0]:
# 2-2. GMM (GPU - RAPIDS cuML)
# RAPIDS cuML의 GPU GMM 사용
gmm = GaussianMixture(n_components=10)
gmm.fit(pca_sampled.cpu().numpy())  # cuML은 Numpy 입력 필요
gmm_cluster_ids_full = gmm.predict(pca_features.cpu().numpy())
gmm_cluster_ids_full = torch.from_numpy(gmm_cluster_ids_full).cuda()


In [0]:
#3. 메타 클러스터링 (앙상블)

# 클러스터 결과 결합 (KMeans + GMM)
meta_features = torch.stack([kmeans_cluster_ids_full, gmm_cluster_ids_full], dim=1).float()

# 메타 KMeans (k=5)
meta_cluster_ids, meta_centers = kmeans(
    X=meta_features,
    num_clusters=5,
    distance='euclidean',
    device=torch.device('cuda'),
    batch_size=512
)

In [0]:
#4. 최종 결과 병합 (PySpark DataFrame)
from pyspark.sql import Row

# 최종 클러스터 ID를 Numpy로 변환
final_cluster_ids_np = meta_cluster_ids.cpu().numpy().astype(int)

# PySpark DataFrame 생성
rows = [Row(id=idx, final_cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)

## 10등분 후 10번 돌리기

In [0]:
#ㅅㅅㅇ
#그냥 스텝와이즈 갈겨버릴껄
from pyspark.ml.clustering import KMeans, BisectingKMeans  # GMM 제거
from pyspark.ml.feature import VectorAssembler

# 1. 데이터 10등분 (randomSplit 사용)
splits = df_pca.randomSplit([0.1] * 10, seed=42)

# 2. 각 분할별로 클러스터링 수행
results = []
for i, split_df in enumerate(splits):
    
    # 2-1. 샘플링 (전체 데이터 대신 분할된 데이터 사용)
    df_sampled = split_df.sample(withReplacement=False, fraction=0.01, seed=42)
    
    # 2-2. 모델 학습 (k=5로 축소, GMM 제외)
    kmeans_model = KMeans(k=5, featuresCol="pca_features").fit(df_sampled)
    bkm_model = BisectingKMeans(k=5, featuresCol="pca_features").fit(df_sampled)
    
    # 2-3. 예측 (전체 분할 데이터에 적용)
    df_kmeans = kmeans_model.transform(split_df).withColumnRenamed("prediction", "kmeans_cluster")
    df_bkm = bkm_model.transform(split_df).withColumnRenamed("prediction", "bkm_cluster")
    
    # 2-4. 결과 병합
    df_ensemble = df_kmeans.select("id", "kmeans_cluster") \
        .join(df_bkm.select("id", "bkm_cluster"), on="id")
    
    # 2-5. 메타 특징 생성
    assembler = VectorAssembler(inputCols=["kmeans_cluster", "bkm_cluster"], outputCol="meta_features")
    df_meta = assembler.transform(df_ensemble)
    
    # 2-6. 메타 클러스터링 (k=5)
    meta_model = KMeans(k=5, featuresCol="meta_features", predictionCol="final_cluster").fit(df_meta)
    df_final = meta_model.transform(df_meta)
    
    results.append(df_final)

# 3. 최종 결과 통합 (옵션)
df_final_cluster = results[0]
for df in results[1:]:
    df_final_cluster = df_final_cluster.union(df)


## 이건 병렬 클러스터링


##인적정보 클러스터링

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

# 인적정보 컬럼 정의
demographic_cols = [
    "남녀구분코드", "연령", "거주시도명", "직장시도명", "가입통신회사코드", "Life_Stage"
]

# 벡터화
assembler_demo = VectorAssembler(inputCols=demographic_cols, outputCol="demo_features")
df_demo = assembler_demo.transform(df)

# 클러스터링 (k=5)
kmeans_demo = KMeans(k=5, featuresCol="demo_features", predictionCol="demo_cluster")
demo_model = kmeans_demo.fit(df_demo)
df_demo_result = demo_model.transform(df_demo)

# 필요시 캐싱 또는 저장
df_demo_result.cache()
df_demo_result.select("id", "demo_cluster").show(10)


2. 라이프스타일/회원상태 클러스터링

In [0]:
# 라이프스타일 컬럼 정의
lifestyle_cols = [
    "마케팅동의여부", "회원여부_연체", "회원여부_이용가능", "동의여부_한도증액안내",
    "수신거부여부_TM", "수신거부여부_메일", "수신거부여부_SMS", "수신거부여부_DM",
    "VIP등급코드", "최상위카드등급코드"
]

# 벡터화
assembler_life = VectorAssembler(inputCols=lifestyle_cols, outputCol="life_features")
df_life = assembler_life.transform(df)

# 클러스터링 (k=5)
kmeans_life = KMeans(k=5, featuresCol="life_features", predictionCol="life_cluster")
life_model = kmeans_life.fit(df_life)
df_life_result = life_model.transform(df_life)

# 필요시 캐싱 또는 저장
df_life_result.cache()
df_life_result.select("id", "life_cluster").show(10)


3. 카드 이용/소비정보 클러스터링

In [0]:
# 소비정보 컬럼 정의
usage_cols = [
    "log_총이용금액", "log_입회경과개월수_신용", "최종카드발급경과월", "탈회횟수_누적",
    "카드신청건수", "소지카드수_유효_신용", "유효카드수_체크", "이용카드수_신용",
    "이용카드수_체크", "청구금액_기본연회비_B0M", "청구금액_제휴연회비_B0M",
    "최종탈회후경과월", "탈회횟수_발급6개월이내", "탈회횟수_발급1년이내"
]

# 벡터화
assembler_usage = VectorAssembler(inputCols=usage_cols, outputCol="usage_features")
df_usage = assembler_usage.transform(df)

# 클러스터링 (k=5)
kmeans_usage = KMeans(k=5, featuresCol="usage_features", predictionCol="usage_cluster")
usage_model = kmeans_usage.fit(df_usage)
df_usage_result = usage_model.transform(df_usage)

# 필요시 캐싱 또는 저장
df_usage_result.cache()
df_usage_result.select("id", "usage_cluster").show(10)


4. 결과 병합 (최종)

In [0]:
# id 기준으로 결과 병합
df_final = df_demo_result.select("id", "demo_cluster") \
    .join(df_life_result.select("id", "life_cluster"), on="id", how="inner") \
    .join(df_usage_result.select("id", "usage_cluster"), on="id", how="inner")

# 최종 결과 확인
df_final.select("id", "demo_cluster", "life_cluster", "usage_cluster").show(10)


# ------ 저장된 테이블 불러오기 & 병합 & 클러스터링& PCA등 시험 --

In [0]:
# 데이터베이스 설정
database_name = "database_pjt"

# 예시 키 컬럼명 (각 테이블마다 실제 컬럼명을 확인해야 함)
join_key = "발급회원번호"  # 또는 customer_id, member_id 등 실명 확인 필요

# 통합할 테이블 목록 (1_fina_로 시작하는 테이블들)
table_names = [
    "1_fina_billing",
    "1_fina_card_history",
    "1_fina_card_usage",
    "1_fina_log",
    "1_fina_member",
    "1_fina_membership",
    "1_fina_marketing_consent"
]

# 테이블 읽고 리스트에 저장
dfs = [
    spark.table(f"{database_name}.{table_name}")
    for table_name in table_names
]

# 테이블 불러오기
dfs_named = {
    table_name: spark.table(f"{database_name}.{table_name}")
    for table_name in table_names
}

# 첫 테이블을 기준으로 Outer Join (1_fina_member 기준 추천)
base_table_name = "1_fina_member"
df_base = dfs_named[base_table_name]

# 나머지 테이블들을 차례대로 outer join
from functools import reduce

tables_to_join = [name for name in table_names if name != base_table_name]

df = reduce(
    lambda df1, name: df1.join(
        dfs_named[name],
        on=join_key,
        how="outer"
    ),
    tables_to_join,
    df_base
)

display(df)


In [0]:
spark.conf.set("spark.rapids.sql.enabled", "true")
spark.conf.set("spark.rapids.sql.explain", "ALL")  # 실행계획에 GPU 사용 여부 출력
spark.conf.set("spark.rapids.sql.exec.BatchScanExec", "true")

In [0]:
df.explain(mode="formatted")